# Ask Sage Python API Client - API Documentation

In this notebook, we will go through the Ask Sage Python Client and review the available endpoints and how to use them.

For more information please visit the Ask Sage Python Client via the link below:

Python API Client: https://pypi.org/project/asksageclient/

Or also reference the Ask Sage Documentation via the this link: https://docs.asksage.ai/

<div class="alert alert-block alert-info">
<b>Tip:</b> Recommend to run the code cells in order to see the output of the endpoints and understand the examples provided. However, be aware that some of the results are post processed to make the output more readable.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> All credential information and API keys are removed from the code cells and declared as environment variables for security purposes. We recommend to do the same when running the code cells if you are sharing the notebook. There is an a file in this repository called `credentials.json.example` as an example of how to structure your credentials file. 
</div>

We will cover all of the following endpoints in this notebook: 

The order of review is based on a logical flow of first understanding what the API has to offer, then interacting with the API to interact with models. 

Also, refer to the Outline of the notebook to navigate to the specific endpoints you are interested in reviewing, but still recommend to run the code cells in order to see the output of the endpoints and understand the examples provided.



## Ask Sage Python API Client Endpoints

<div style="text-align: center;">

|       Function Name         |                       Description                     |
|:---------------------------:|:-----------------------------------------------------:|
|         `get_models`         | Get the available models from the Ask Sage service    |
|       `add_dataset`         |                   Adds a new dataset                  |
|     `delete_dataset`        |              Deletes a specified dataset              |
|     `assign_dataset`        |                   Assigns a dataset                   |
|     `get_user_logs`         |             Retrieves all logs for user               |
|    `get_user_logins`        | Retrieves login information for a specific user       |
|          `query`            | Interact with the /query endpoint of the Ask Sage API |
|    `query_with_file`        |         Executes a query using a file                 |
|      `query_plugin`         | Executes a query using a specific plugin              |
|    `execute_plugin`         | Executes a plugin with the provided content           |
| `follow_up_questions`       | Interact with the /follow-up-questions endpoint of the Ask Sage API |
|        `tokenizer`          | Interact with the /tokenizer endpoint of the Ask Sage API |
|      `get_personas`         | Get the available personas from the Ask Sage service  |
|      `get_datasets`         | Get the available datasets from the Ask Sage service  |
|       `get_plugins`         | Get the available plugins from the Ask Sage service   |
|  `count_monthly_tokens`     | Get the count of monthly training tokens spent for this user from the Ask Sage service |
|`count_monthly_teach_tokens` | Counts the number of teach tokens used in a month     |
|          `train`            | Train the model based on the provided content         |
|    `train_with_file`        | Train the dataset based on the provided file          |
|          `file`             | Upload a file to the Ask Sage service                 |

</div>


There are 20 endpoints available in the Ask Sage Python API Client. We will go through each of them in detail.

# Environment Setup 

##  A. Import Libraries and Set Environment Variables 

Let's start by importing the necessary libraries and setting the environment variables.

<div class="alert alert-block alert-warning">
<b>Note:</b> There will be more detail within this notebook than what is relevant to most users, but it is important to understand the different endpoints and what they represent. Also, note that the code is not being written in a production-ready manner, but rather to demonstrate the different endpoints and their outputs. For production, you would want to write more robust code with error handling, security, and much more. 
</div>




### Python Libraries


In [1]:
# === Core Python Modules ===
import os         # Interact with the operating system (file management, paths)
import re         # Work with regular expressions
import time       # Time-related functions
import random     # Generate random numbers
import json       # Work with JSON data
import pprint     # Pretty-print complex data structures
from datetime import datetime, timedelta  # Date and time utilities

# === Type Hints ===
from typing import List, Tuple, Optional, Dict, Union  # Added Dict and Union for broader type hinting

# === External Libraries ===
import requests        # Send HTTP requests
import pandas as pd     # Data manipulation and analysis
import panel as pn      # Build interactive web applications
import plotly.express as px   # Simple Plotly visualizations
import plotly.graph_objects as go  # Advanced Plotly visualizations
import matplotlib.pyplot as plt   # Static, animated, and interactive visualizations
from PIL import Image      # Image processing
from faker import Faker    # Generate fake data for testing

# === Custom Modules ===
from asksageclient import AskSageClient  # Interact with the AskSage API

# === Panel Setup ===
pn.extension('plotly')  # Initialize Panel with Plotly support

# === Future Imports ===
from __future__ import annotations  # Ensure compatibility with future Python versions

# === Logging Setup ===
import logging  # Added logging for better debugging and monitoring
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') 

# === Path Utilities ===
from pathlib import Path  # Manage file paths effectively



### Accessing Environment Variables (Credentials)



In [2]:
# Function to load credentials from a JSON file
def load_credentials(filepath: str) -> dict:
    """
    Load credentials from a JSON file.

    Args:
        filepath (str): Path to the credentials file.

    Returns:
        dict: Parsed credentials data.

    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the file contains invalid JSON.
        KeyError: If required keys are missing in the credentials.
    """
    try:
        with open(filepath, 'r') as file:
            credentials = json.load(file)
            # Validate required keys
            if 'credentials' not in credentials or 'api_key' not in credentials['credentials']:
                raise KeyError("Missing required keys in the credentials file.")
            return credentials
    except FileNotFoundError:
        raise FileNotFoundError(f"Credentials file not found at: {filepath}")
    except json.JSONDecodeError:
        raise ValueError(f"Invalid JSON format in the credentials file: {filepath}")

# Load the credentials
credentials = load_credentials('../../credentials.json')

# Extract the API key and email from the credentials
api_key = credentials['credentials']['api_key']
email = credentials['credentials']['Ask_sage_user_info']['username']


## B. Setup Ask Sage Python API Client

The AskSage Python API Client provides a simple way to interact with the AskSage API. The client provides methods for each endpoint, making it easy to use the API.

First order is defining the client and setting the credentials for the client, which are the email and API key.

Note: Users have the option to generate a token for the API key, but this is not required. The API key is sufficient to access the API. However, they API key does not expire, thus take precautions to keep it secure.


<div class="alert alert-block alert-warning">
<b>Note:</b> If you are not obtaining your API key from https://chat.asksage.ai/ and are on other tenant/instance of Ask Sage, please update the base URL within the Ask Sage Client package installed. See image below for where to update the base URL in the package.

![Ask Sage Client Base URL](/Users/mark.espinozaasksage.ai/Code/AskSage-Open-Source-Community/images/update_base_url.png)
</div>



In [3]:
"""  
class AskSageClient(
    email: email, # The email address of the user
    api_key: api_key, # The API key for the AskSage API, which can be obtained from the AskSage website
    user_base_url: str = 'https://api.asksage.ai/user', # The base URL for the user API
    server_base_url: str = 'https://api.asksage.ai/server' # The base URL for the server API
)
"""

ask_sage_client = AskSageClient(email, api_key) # Create an instance of the AskSageClient class with the email and api_key 

# Python Client Endpoints Review

## 1. Retrieve Available Models
The `get_models` function fetches a list of all available models provided by Ask Sage. These models can be utilized with the `query` function to process user queries. Developers should familiarize themselves with each model's capabilities and differences to ensure effective usage.


In [4]:
# Fetch the list of models available for the user
response = ask_sage_client.get_models()

# Check if the response contains the expected 'response' key
if 'response' in response:
    models = response['response']
    
    # Convert the models list into a DataFrame for better visualization
    models_df = pd.DataFrame(models)
    print('Models available for the user on the respetive Ask Sage Tenant/Account:')
    print('--------------------------------------------------')
    display(models_df)
    


Models available for the user on the respetive Ask Sage Tenant/Account:
--------------------------------------------------


,0
0,aws-bedrock-titan
1,aws-bedrock-claude-35-sonnet-gov
2,llma3
3,claude2
4,claude-3-opus
5,claude-3-sonnet
6,claude-35-sonnet
7,claude-37-sonnet
8,cohere
9,mistral-large


<div class="alert alert-block alert-warning">
<b>Note:</b> Models available to users can be limited/restricted if they are part of a enterprise account. Admins/account owners can restrict access to certain models.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> For more information on the models, please refer to the Ask Sage `Help` section in the platform and navigate to the `Models` tab. 
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> The list of models provided in the output are the names of which users will need to define using other endpoints such as `query`, where model = 'model_name'. (e.g. model = 'gpt-4o-mini-gov', model = 'aws-bedrock-claude-35-sonnet-gov', etc.)
</div>

<div class="alert alert-block alert-danger">
<b>WARNING:</b> Models do become deprecated and are removed from the list of available models. If you are using a model that is deprecated, you will need to update your code to use a different model. Especially for models that are not open source.
</div>


## 2. Get User Logs

This endpoint returns the last 100 prompts of the user. 

There are no parameters required for this endpoint. 

The output will be the following:
- All the prompts that the user has made.
- completion_tokens: The completion tokens of the prompt.
- date_time: The date and time of the prompt.
- id: The ID of the prompt.
- ip: The IP address of the prompt.
- model: The model used for the prompt.
- prompt: The prompt text.
- prompt_tokens: The prompt tokens.
- response: The response text.
- teach: The teach flag (true or false).
- total_tokens: The total tokens 


<div class="alert alert-block alert-warning">
<b>Note:</b> Uncomment the `display` call to see the output of the endpoint in the cell below. It was intentionally commented out to avoid showing the output in the notebook. 

Also, be aware that this endpoint outputs the last 100 prompts that the user has made, which may contain sensitive information. Therefore, it is recommended to use this endpoint with caution and avoid displaying the output in a shared notebook.
</div>

<div class="alert alert-block alert-danger">
<b>WARNING:</b> The output of this endpoint could contain sensitive information. Please be cautious when displaying the output in a shared notebook. 
</div>




In [5]:
# Fetch user logs using the AskSage client
user_logs = ask_sage_client.get_user_logs()

# print(user_logs)  # Uncomment this line to see the raw user logs. 

# Check if logs are available and display them
if user_logs and 'response' in user_logs:
    print(f"Total logs retrieved: {len(user_logs['response'])}")
    # display(pd.DataFrame(user_logs['response']).head(10))  # Display the first 10 logs in a DataFrame - Uncomment this line to see the logs
else:
    print("No user logs found or response key missing.")

Total logs retrieved: 100


## 3. Get User Logins

This endpoint returns the last logins of the user. By default, it returns the last 5 logins.

The only parameter required for this endpoint is the 'limit' parameter, which specifies the number of logins to return. The default value is 5 and the maximum value is 100.

So in return, one will get the following information: 
- date_time: The date and time of the login.
- id: The ID of the login.
- ip: The IP address of the login.
- status: The status of the login (success or failure).
- type: endpoint type (login).
- status_code: The status code of the login (200 for success, 400 for failure).

<div class="alert alert-block alert-warning">
<b>Note:</b> Data is intentionally replaced with dummy data for security purposes and to avoid showing sensitive information (e.g. IP address, status code, ID, etc.).
</div>

The example provided is modified to show the output of the endpoint in a clean dataframe format - note we intentionally replaced the actual output with a dummy output for security purposes. 


In [6]:
# Fetch user login data using the AskSage client
user_logins = ask_sage_client.get_user_logins(limit=50)

# Check if the response contains login data
if user_logins and 'response' in user_logins:
    # Extract relevant fields and replace sensitive data with mock values
    extracted_data = [
        {
            'date_time': entry['date_time'],
            'ip_address': '.'.join([str(random.randint(0, 255)) for _ in range(4)]),  # Mock IP address for privacy
            'id': '111111',  # Mock ID for privacy
            'status': entry['status'],
            'type': entry['type']
        }
        for entry in user_logins['response']
    ]

    # Convert extracted data into a DataFrame for better visualization
    user_logins_df = pd.DataFrame(extracted_data)
    display(user_logins_df)
else:
    print("No user login data found or response key missing.")

,date_time,ip_address,id,status,type
0,"Thu, 08 May 2025 21:42:04 GMT",56.171.16.40,111111,Success,get_token_with_api_key
1,"Thu, 08 May 2025 21:34:15 GMT",115.89.15.69,111111,Success,get_token_with_api_key
2,"Thu, 08 May 2025 21:26:33 GMT",116.157.217.231,111111,Success,get_token_with_api_key
3,"Thu, 08 May 2025 18:30:50 GMT",121.150.99.242,111111,Success,get_token_with_api_key
4,"Thu, 08 May 2025 16:48:53 GMT",57.120.247.18,111111,Success,get_token_with_api_key
5,"Thu, 08 May 2025 15:44:32 GMT",110.194.177.232,111111,Success,get_token_with_api_key
6,"Thu, 08 May 2025 15:43:08 GMT",130.134.193.133,111111,Success,login_with_mfa_2
7,"Thu, 08 May 2025 15:42:33 GMT",208.207.236.203,111111,Success,login_with_mfa_1
8,"Thu, 08 May 2025 15:26:25 GMT",30.80.30.128,111111,Success,get_token_with_api_key
9,"Thu, 08 May 2025 15:08:03 GMT",228.250.203.82,111111,Success,get_token_with_api_key


## 4. Count Monthly Tokens

Get the count of monthly `inference tokens` spent in the current month.

--------------------------------

### Additional Information on Tokens within Ask Sage

<b> Ask Sage Tokens Overview </b>

Ask Sage tokens are the primary means of accessing the Ask Sage platform and function as its currency. In the context of Generative Artificial Intelligence (GenAI), a token represents a unit of text that the model processes. Tokens can vary in length, ranging from a single character to an entire word, depending on the language and the model's implementation.

<b> Token Allocation </b>

For users of Ask Sage, tokens are allocated based on their subscription plan and can be used for various platform features, such as training models and making inferences. There are two main categories of tokens:

1. Inference Tokens
   - **Purpose**: Used to make predictions or inferences using the models available on the platform.
   - **Consumption**: These tokens are consumed when users submit prompts or interact with the models in any way that requires processing text data.

2. Training Tokens
   - **Purpose**: Used to create training datasets in the Ask Sage platform.
   - **Consumption**: These tokens are consumed when users upload data to a dataset for training purposes.


Reference the following link for more information on Ask Sage tokens: https://docs.asksage.ai/docs/asksage-platform/%20asksage-tokens.html

<b> Token Replenishment <b> 

Tokens replenish automatically on the first day of each month. Users can also purchase additional tokens if needed.

--------------------------------


In [7]:
def get_monthly_token_usage(client):
    """
    Retrieves and prints the count of monthly inference/query tokens used.

    Args:
        client: The Ask Sage client instance.

    Returns:
        int: The count of monthly inference/query tokens used.
    """
    count_monthly_usage = client.count_monthly_tokens()  # Get the count of monthly tokens used
    count = count_monthly_usage['response']  # Extract the count from the response
    print(f"The count of monthly inference/query tokens used are: {count}")
    return count

count_start = get_monthly_token_usage(ask_sage_client)


The count of monthly inference/query tokens used are: 1418083


## 5. Count Monthly Teach Tokens

Get the count of monthly `teach tokens` spent in the current month. `teach tokens` are the same as `training tokens` and are used to create `Ask Sage datasets`.

`Ask Sage datasets` are used to perform RAG(Retrieval-Augmented Generation) where the model retrieves information from the dataset to generate responses leveraging sophisticated semantic search capabilities.

In [8]:
def get_monthly_teach_token_usage(client):
    """
    Retrieves and prints the count of monthly teach/training tokens used.

    Args:
        client: The Ask Sage client instance.

    Returns:
        int: The count of monthly teach/training tokens used.
    """
    count_monthly_usage = client.count_monthly_teach_tokens()  # Get the count of monthly teach tokens used
    count_training_tokens_starting = count_monthly_usage['response']  # Extract the count from the response
    print(f"The count of monthly teach/training tokens used are: {count_training_tokens_starting}")
    return count_training_tokens_starting

count_training_tokens_starting = get_monthly_teach_token_usage(ask_sage_client)



The count of monthly teach/training tokens used are: 49480


#### Extra Function - Combination of Monthly Tokens and Teach Tokens Call

See below function to get the count of monthly and teach tokens spent - this will be used throughout the notebook to get the monthly tokens and teach tokens spent as one progresses through the notebook: 
- Comparing the monthly tokens and teach tokens spent at start versus current stage of the notebook.
- Comparing the monthly tokens and teach tokens spent at the current stage of the notebook versus the previous stage of the notebook.

In [9]:
def collect_token_counts(client):
    """
    Returns a dict with the exact structure expected by the reporting
    function, using the two existing helpers above.
    """
    return {
        "teach_tokens": get_monthly_teach_token_usage(client),
        "query_tokens": get_monthly_token_usage(client)
    }

# ------------------------------------------------------------------
# Global state
# ------------------------------------------------------------------
token_counts_start = None    # first snapshot taken the first time we run
token_usage_history = []     # list of {"start":…, "end":…} per stage
stage_counter = 0


# ------------------------------------------------------------------
#  Reporting function – now calls collect_token_counts()
# ------------------------------------------------------------------
def print_token_volume_report(client):
    """
    Prints a multi-stage token-usage report that compares the very first
    snapshot, the previous stage, and the current end of script.
    """
    global token_counts_start, token_usage_history, stage_counter

    stage_counter += 1
    print(f"\n📊 Token Volume Report 📊 (Stage #{stage_counter})")
    print("--------------------------------------------------")

    # --- 4-a. First-run initialization ---------------------------------
    if token_counts_start is None:
        token_counts_start = collect_token_counts(client)
        print("🔹 Token volume at the *start* of script recorded.")
    else:
        print("🔹 Reusing token volume from script start.")

    # --- 4-b. Current end-of-stage snapshot ----------------------------
    token_counts_end = collect_token_counts(client)
    print("🔹 Token volume recorded *after current script execution*.")
    print("--------------------------------------------------")

    # --- 4-c. Store history --------------------------------------------
    token_usage_history.append({
        "start": token_counts_start,
        "end"  : token_counts_end
    })

    # --- 4-d. Differences vs prior end ---------------------------------
    if len(token_usage_history) > 1:
        prior_end   = token_usage_history[-2]["end"]
        prior_stage = stage_counter - 1
        print(f"🔹 Change from Prior End (Stage #{prior_stage}) → "
              f"Current End (Stage #{stage_counter}):")
        print({
            "teach_tokens_diff": token_counts_end["teach_tokens"]
                                 - prior_end["teach_tokens"],
            "query_tokens_diff": token_counts_end["query_tokens"]
                                 - prior_end["query_tokens"],
        })
    else:
        print("🔹 No prior stage available for difference comparison.")

    # --- 4-e. Pretty tables --------------------------------------------
    start_df = pd.DataFrame([token_counts_start],
                            index=[f"Start (Stage #1)"])
    end_df   = pd.DataFrame([token_counts_end],
                            index=[f"End (Stage #{stage_counter})"])
    frames   = [start_df]
    if len(token_usage_history) > 1:
        prior_df = pd.DataFrame([prior_end],
                                index=[f"Prior (Stage #{prior_stage})"])
        frames.append(prior_df)
    frames.append(end_df)

    print("🔹 Token Usage Table:")
    print(pd.concat(frames))
    print("--------------------------------------------------")

    # --- 4-f. Comparison flags -----------------------------------------
    def diff_flags(a, b):
        return {
            "teach_tokens_diff": a["teach_tokens"] - b["teach_tokens"],
            "query_tokens_diff": a["query_tokens"] - b["query_tokens"],
            "teach_tokens_flag": "✅ Changed" if a["teach_tokens"] != b["teach_tokens"]
                                 else "❌ No Change",
            "query_tokens_flag": "✅ Changed" if a["query_tokens"] != b["query_tokens"]
                                 else "❌ No Change"
        }

    if len(token_usage_history) > 1:
        comparison_df = pd.DataFrame({
            f"Start vs End (1 → {stage_counter})":
                diff_flags(token_counts_end, token_counts_start),
            f"Prior vs End ({prior_stage} → {stage_counter})":
                diff_flags(token_counts_end, prior_end)
        }).T
    else:
        comparison_df = pd.DataFrame({
            f"Start vs End (1 → {stage_counter})":
                diff_flags(token_counts_end, token_counts_start)
        }).T

    print("📈 Token Volume Comparison Flags:")
    display(comparison_df)
    print("--------------------------------------------------")

    # --- 4-g. Volume used during this stage (relative to start) -------
    stage_usage = {
        "teach_tokens_used": token_counts_end["teach_tokens"] - token_counts_start["teach_tokens"],
        "query_tokens_used": token_counts_end["query_tokens"] - token_counts_start["query_tokens"]
    }

    print("📦 Total Token Volume Used Since Script Start:")
    print(stage_usage)
    print("==================================================")

    # --- 4-h. Volume used in this specific stage only -------------------
    if len(token_usage_history) > 1:
        current_stage_start = token_usage_history[-2]["end"]
    else:
        current_stage_start = token_counts_start  # Fall back to start if this is stage 1

    current_stage_usage = {
        "teach_tokens_used_this_stage": token_counts_end["teach_tokens"] - current_stage_start["teach_tokens"],
        "query_tokens_used_this_stage": token_counts_end["query_tokens"] - current_stage_start["query_tokens"]
    }

    print("📦 Token Volume Used in This Stage Only:")
    print(current_stage_usage)
    print("==================================================")


# ------------------------------------------------------------------
# ------------------------------------------------------------------
print_token_volume_report(ask_sage_client)


📊 Token Volume Report 📊 (Stage #1)
--------------------------------------------------
The count of monthly teach/training tokens used are: 49480
The count of monthly inference/query tokens used are: 1418083
🔹 Token volume at the *start* of script recorded.
The count of monthly teach/training tokens used are: 49480
The count of monthly inference/query tokens used are: 1418083
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 No prior stage available for difference comparison.
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
End (Stage #1)           49480       1418083
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 1),0,0,❌ No Change,❌ No Change


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 0, 'query_tokens_used': 0}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 0}


## 6. Get Personas

This endpoint returns the available personas from the Ask Sage service. Users will have access to all the standard personas that are available in the Ask Sage service for all paid users, and any custom personas that have been created in the platform. 

<div class="alert alert-block alert-warning">
<b>Note:</b> Custom personas can only be created by users within the Ask Sage platform and not through the API.
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> Recommend saving any system prompts as personas in the platform for future use. This will allow users to easily access the system prompts and use them in the platform. However, when using the API, we recommend using the `system_prompt` parameter instead of the `persona` parameter to have more control over the system prompt being used.
</div>

<div class="alert alert-block alert-info">
<b>Tip:</b> The 'id' provided in the output of the endpoint for each persona is unique. When defining a persona in the `query` endpoint or any other endpoint that has the option, the user will need to use the 'id' of the persona not the name of the persona or else it will automatically default to the default persona. Example: persona = '1' or persona = '2' etc. 
</div>



In [10]:
# Get the personas using the get_personas endpoint
get_personas = ask_sage_client.get_personas()

# Extract relevant information from the response
def extract_personas(response):
    return [
        {
            'id': persona['id'],
            'name': persona['name'],
            'description': persona['description']
        }
        for persona in response['response']
    ]

# Extract the personas using the extract_personas function
personas = extract_personas(get_personas)

# Putting all information into a DataFrame
personas_df = pd.DataFrame(personas)

# Remove the index column for display
personas_df.index = personas_df.index.map(lambda _: '')

# Set column width to display full content and enable word wrapping
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
display(personas_df)

,id,name,description
,1,Ask Sage,"Use this persona when you need a general-purpose AI that can handle a wide range of tasks, from translating languages to writing essays and code."
,2,Legal Assistant,Use this persona when you need legal advice or information. This persona can provide accurate and helpful advice on a wide range of legal topics.
,3,Contracting Officer,"Use this persona when you have questions or need advice about government contracts, Federal Acquisition Regulation (FAR) regulations, the Defense Federal Acquisition Regulation Supplement (DFARS) regulations and acquisition related questions."
,4,Software Developer,"Use this persona when you need to write, review, or debug code. This persona can also provide advice on software development best practices and security standards."
,5,ISSO (Cyber),Use this persona when you need advice or information about cybersecurity requirements and issues. This persona can provide accurate and helpful advice on a wide range of cybersecurity topics.
,6,System Administrator,"Use this persona when you need advice or information about system designs, security, and issues. This persona can provide accurate and helpful advice on a wide range of system administration topics."
,7,Structural Engineer,Use this persona when you need advice or information about structural engineering. This persona can provide accurate and helpful advice on a wide range of structural engineering topics.
,8,Electrical Engineer,Use this persona when you need advice or information about electrical engineering. This persona can provide accurate and helpful advice on a wide range of electrical engineering topics.
,9,Accountant,Use this persona when you need advice or information about financial matters. This persona can provide accurate and helpful advice on a wide range of accounting and financial topics.
,10,DevSecOps Engineer,Use this persona when you need advice or information about DevSecOps and software operations. This persona can provide accurate and helpful advice on a wide range of DevSecOps topics.


## 7. Tokenizer

This endpoint is used to interact with the `/tokenizer` endpoint of the Ask Sage API. It is used to tokenize the text data provided to the model. 

A token is a unit of text that the model processes. Tokens can vary in length, ranging from a single character to an entire word, depending on the language and the model's implementation. A good rule of thumb is 1 token is equivalent to 3.7 english characters.

<div class="alert alert-block alert-warning">
<b>Note:</b> The `tokenizer` endpoint is useful in estimating the number of tokens that will be used for a given text. This can be helpful in managing the number of tokens used when prompting a model. Key is it's an estimate and not an exact number. 
</div>

In the example provided below, we tokenize 3 different texts to see the number of tokens used for each text. Feel free to modify the text to see the number of tokens used for different text inputs. 



In [11]:
texts = [
    "This endpoint is used to interact with the /tokenizer endpoint of the Ask Sage API. It is used to tokenize the text provided to the model. The tokenizer is used to split the text into tokens that the model can understand and process. The tokenizer is used to preprocess the text before it is passed to the model for processing. Thus in return the output will be the tokens of the text provided.",
    "Ask Sage is a GenAI Platform.",
    "{ sdfd < > ? / . , ; : ' \" [ ] { } ( ) }"
]

total_tokens = 0

for text in texts:
    tokenize_data = ask_sage_client.tokenizer(text)  # Tokenize the text using the tokenizer method
    # Uncomment the next line to display the raw response
    # display(tokenize_data)  
    tokens_count = tokenize_data['response']  # Assuming 'response' contains the token count
    total_tokens += int(tokens_count)  # Accumulate the total number of tokens
    display(f'The number of tokens in the text is: {tokens_count} Tokens')

display(f'Total number of tokens across all texts: {total_tokens} Tokens')

'The number of tokens in the text is: 84 Tokens'

'The number of tokens in the text is: 13 Tokens'

'The number of tokens in the text is: 24 Tokens'

'Total number of tokens across all texts: 121 Tokens'

In [12]:
print_token_volume_report(ask_sage_client)


📊 Token Volume Report 📊 (Stage #2)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 49480
The count of monthly inference/query tokens used are: 1418083
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #1) → Current End (Stage #2):
{'teach_tokens_diff': 0, 'query_tokens_diff': 0}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #1)         49480       1418083
End (Stage #2)           49480       1418083
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 2),0,0,❌ No Change,❌ No Change
Prior vs End (1 → 2),0,0,❌ No Change,❌ No Change


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 0, 'query_tokens_used': 0}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 0}


## 8. File Endpoint

The `file` endpoint is designed for extracting information from files, enabling seamless ingestion into Ask Sage. Below is a detailed overview of its functionality, parameters, and usage.

### Parameters:
- **`file_path` (str)**:  
  Specifies the path to the file you want to upload. This should be a string representing the full file path.

- **`strategy` (str, optional)**:  
  Defines the parsing strategy to be used. Available options include:  
  - `'auto'`: Default strategy that automatically selects the optimal parsing method.  
  - `'fast'`: A quicker parsing method that may trade off some accuracy.  
  - `'hi_res'`: A slower, high-accuracy method using Optical Character Recognition (OCR), ideal for image-based files.

### Returns:
The method returns a dictionary containing:
- **`status`**: Indicates the success or failure of the operation.
- **`data`**: Includes any extracted information from the file.

---

### Usage:
The `file` endpoint is highly versatile for parsing and extracting text from various file types, such as PDFs, images, and documents. The extracted text can then be utilized in other endpoints, such as:
- **`query_with_file`**: For querying extracted content.
- **`train_with_file`**: For training datasets using the extracted content.

This endpoint is particularly useful when working with Ask Sage **Plugins/Agents** to automate workflows involving file ingestion and processing.

---

### Key Notes:
1. **Data Ingestion**:  
   The `file` endpoint is integral to ingesting data into Ask Sage. For example, you can parse a file and use the `Train` endpoint to assign its content to a dataset within Ask Sage.


2. **Supported File Types**:  
   Ask Sage supports a variety of file types. For a complete list, refer to the [Ask Sage Documentation](https://docs.asksage.ai/) or check directly within the Ask Sage platform. If you encounter issues with unsupported file types, contact Ask Sage support for assistance.

3. **Image & Audio Files**:  
   The `file` endpoint can also handle image and audio files, however when processing these files users will be charged for the inference tokens used to extract context from them. Therefore, image and audio files are more expensive to process than text files since they require inference tokens to extract the context from them.

---

### Example Workflow:
1. Use the `file` endpoint to parse a file and extract its content.
2. Train or update embedding Ask Sage datasets with the extracted content via the `Train` endpoint.
3. Perform retrieval-augmented generation (RAG) on the Ask Sage dataset with the embedding using the `query` endpoint.

1. Parse a file using the `file` endpoint to extract its content.
2. Combine the extracted content with a {prompt} and generate a response using the `query` endpoint.

By understanding the `file` endpoint, you gain insight into how data ingestion works within Ask Sage, enabling efficient use of its services.

In [13]:
# data path
path = 'data/query_with_file/'

# get files in the data path
files = os.listdir(path)

# iterate over each file in the directory
for file in files:
    # get the file endpoint
    file_endpoint = ask_sage_client.file(file_path=path + file, strategy='auto')
    
    # display the full response for the current file
    display(file_endpoint)
    
    # extract the 'ret' field from the response
    ret = file_endpoint['ret']
    
    # count # of chunks by the number of 'asksage_metadata' appears in ret
    count = ret.count('asksage_metadata')
    print(f'The number of chunks in {file} is: {count}')

{'response': 'OK',
 'ret': '{"asksage_metadata": {"filename": "random_photo_2.png", "page_number": "1"}}\nThe image depicts a beautifully arranged charcuterie board placed on a wooden serving tray with black handles. The board is filled with a variety of snacks, cheeses, meats, vegetables, and dips, creating a colorful and appetizing display. Below is a detailed description of the components:\n\n### **Top Section:**\n1. **Cheese and Meat Arrangement:**\n   - Slices of white cheese are arranged in a semi-circle pattern.\n   - Thinly folded slices of cured meat (likely salami or prosciutto) are placed next to the cheese, forming a decorative fan-like arrangement.\n\n2. **Three-Compartment Bowl:**\n   - **Top Left Compartment:** Contains a mix of green and black olives.\n   - **Top Right Compartment:** Contains marinated mozzarella balls sprinkled with herbs and spices.\n   - **Bottom Compartment:** Contains hummus topped with a dollop of red chili sauce or salsa.\n\n### **Middle Section:

The number of chunks in random_photo_2.png is: 1


{'response': 'OK',
 'ret': '{"asksage_metadata": {"filename": "ghi-780.png", "page_number": "1"}}\nThe image depicts a group of construction workers actively engaged in building the wooden framework of a house. The scene is outdoors, surrounded by trees, suggesting the construction site is in a rural or suburban area. The workers are wearing safety gear, including hard hats in various colors such as white, yellow, and green, which may indicate different roles or teams.\n\nThe structure being built appears to be a single-story house with a pitched roof. The workers are handling wooden trusses and beams, which form the skeleton of the roof. Some workers are standing on ladders or elevated platforms to position and secure the trusses, while others are on the ground lifting and stabilizing the beams. The teamwork is evident as they coordinate their efforts to align and assemble the framework.\n\nThe foundation of the house is visible, made of concrete, and the walls are constructed with wo

The number of chunks in ghi-780.png is: 1


{'response': 'OK',
 'ret': '\n{"asksage_metadata": {"filename": "LORA- LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS.pdf", "page_number": 1}}\n1 2 0 2\nt c O 6 1\n] L C . s c [\n2 v 5 8 6 9 0 . 6 0 1 2 : v i X r a\nLORA: LOW-RANK ADAPTATION OF LARGE LAN- GUAGE MODELS\nEdward Hu∗ Yuanzhi Li Microsoft Corporation {edwardhu, yeshe, phwallis, zeyuana, yuanzhil, swang, luw, wzchen}@microsoft.com yuanzhil@andrew.cmu.edu (Version 2)\nYelong Shen∗ Shean Wang\nPhillip Wallis\nZeyuan Allen-Zhu\nLu Wang\nWeizhu Chen\nABSTRACT\nAn important paradigm of natural language processing consists of large-scale pre- training on general domain data and adaptation to particular tasks or domains. As we pre-train larger models, full ﬁne-tuning, which retrains all model parameters, becomes less feasible. Using GPT-3 175B as an example – deploying indepen- dent instances of ﬁne-tuned models, each with 175B parameters, is prohibitively expensive. We propose Low-Rank Adaptation, or LoRA, which freezes the pre- tra

The number of chunks in LORA- LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS.pdf is: 26


{'response': 'OK',
 'ret': '{"asksage_metadata": {"filename": "random_photo.png", "page_number": "1"}}\nThe image depicts a black and white dog, likely a Border Collie or a similar breed, lying on a carpeted floor. The dog\'s body is mostly black, with white markings on its chest, legs, and face. Its tail is black with a white tip, and its ears are upright, giving it an alert appearance. The dog is wearing a silver chain collar around its neck.\n\nThe dog is lying on its side, with its front legs crossed in a relaxed manner. Its head is turned slightly upward, and its gaze is directed toward the camera, showing one blue eye and one brown eye, which is a striking feature. The dog\'s nose is pink with black spots.\n\nNear the dog, there are several objects visible:\n1. A bright orange and blue rubber ball is positioned near the dog\'s neck, partially under its head.\n2. A spiked blue and white toy is on the carpet, slightly above the dog.\n3. A pair of dumbbells, one marked with "35" (in

The number of chunks in random_photo.png is: 1


{'response': 'OK',
 'ret': '\n{"asksage_metadata": {"filename": "In the small village of Willowbrook.pdf", "page_number": 1}}\nIn the small village of Willowbrook, there lived a scruffy little dog named Toby who had an unusual dislike for cats. Toby, with his ruffled fur and eager eyes, was known around the village for his playful spirit, but he would always steer clear of any feline. One day, a new cat moved into the neighborhood. The cat, sleek and confident, quickly noticed Toby\'s aversion. Curious and a bit mischievous, the cat decided to find out why Toby was so hesitant around its kind. This set the stage for a series of amusing encounters, as the cat tried to win over the reluctant Toby, leading to unexpected friendship and adventures that would change Toby\'s mind about cats forever.\n',
 'sent_filename': 'InthesmallvillageofWillowbrook.pdf',
 'status': 200}

The number of chunks in In the small village of Willowbrook.pdf is: 1


In [14]:
print_token_volume_report(ask_sage_client)


📊 Token Volume Report 📊 (Stage #3)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 49480
The count of monthly inference/query tokens used are: 1420075
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #2) → Current End (Stage #3):
{'teach_tokens_diff': 0, 'query_tokens_diff': 1992}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #2)         49480       1418083
End (Stage #3)           49480       1420075
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 3),0,1992,❌ No Change,✅ Changed
Prior vs End (2 → 3),0,1992,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 0, 'query_tokens_used': 1992}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 1992}


The reason this endpoint used tokens is that image and audio files require inference tokens to process. Therefore, the user will be charged for the inference tokens used when processing these files.

## 9. Get Datasets

The `get_datasets` function retrieves all the available `Ask Sage Datasets` that have been created by the user/account. These datasets are vector datasets that are used to perform RAG(Retrieval-Augmented Generation) where the model retrieves information from the dataset to generate responses leveraging sophisticated semantic search capabilities brought to you by Ask Sage. 

Access to custom datasets that are created on the account are only available to the user that created the dataset unless they share the dataset with other users. Likewise, if someone else shares a dataset with you, you will be able to see the dataset in your account. 

When a dataset is shared there are various permissions that can be set for the dataset, such as `read`, `edit`, `Admin` access. 

- `read`: Allows the user to select the dataset for use for querying - RAG. 
- `edit`: Allows the user to edit the dataset by being able to view the content and add more content to the dataset.
- `Admin`: Allows the user to also edit the dataset and share the dataset with other users.

Only the owner of the dataset can delete the dataset and delete the content within the dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> Sharing a dataset is easily done within the Ask Sage platform by navigating to `Data Management` and selecting the dataset you want to share. (There is a share icon next to the dataset name).

Also, elevating a user to anything other than `read` access can only be done within the Ask Sage platform and not through the API.
</div>


<center>

| Index | Dataset Name              |
|-------|---------------------------|
| 0     | Acquisition.gov           |
| 1     | Air Force                 |
| 2     | DoD                       |
| 3     | Department of Defense     |
| 4     | Learn with Nic            |
| 5     | In the Nic of Time        |
| 6     | Platform One              |
| 7     | Nic Chaillan's Website    |
| 8     | Cloud One                 |
| 9     | NIST_NVD_CVE              |
| 10    | Sage                      |
| 11    | user_custom_1514845_datasetname_content  |
</center>

<div class="alert alert-block alert-warning">
<b>Note:</b>Custom datasets follow a specific naming convention, as demonstrated in the example above. The structure is outlined below:

- **Prefix**: `user_custom_`
- **ID**: A unique identifier, e.g., `1514845`
- **Name**: The dataset name, e.g., `datasetname`
- **Suffix**: `_content`

When combined, the full dataset name becomes:  
`user_custom_1514845_datasetname_content`

This naming convention is essential for identifying datasets and their content. When using a dataset as a parameter in endpoints like `query` or `train`, ensure you provide the full dataset name in the format shown above.
</div>


In [15]:
# extract relevant information from the response
def extract_datasets(response):
    return response['response']

get_datasets = ask_sage_client.get_datasets() # Get the datasets using the get_datasets endpoint


def display_datasets(ask_sage_client):
    """   
    Function to display the datasets in a dataframe

    Parameters:
    ask_sage_client: AskSageClient - The AskSageClient instance

    Returns:
    None - Displays the datasets in a dataframe
    
    """
    get_datasets = ask_sage_client.get_datasets() # Get the datasets using the get_datasets endpoint
    datasets = extract_datasets(get_datasets) # Extract the datasets using the extract_datasets function
    datasets_df = pd.DataFrame(datasets)
    return datasets_df

user_datasets = display_datasets(ask_sage_client) # Display the datasets in a dataframe
display(user_datasets)


,0
0,Acquisition.gov
1,Air Force
2,DoD
3,Department of Defense
4,Learn with Nic
5,In the Nic of Time
6,Platform One
7,Nic Chaillan's Website
8,Cloud One
9,NIST_NVD_CVE


## 10. Add Dataset

This endpoint is used to create a new `Ask Sage dataset`, once the dataset is created via the API it will also be available on the AskSage platform for the user to use. 

The `add_dataset` function creates a new dataset in Ask Sage. The dataset can be used to store and manage data for utilizing retrieval-augmented generation (RAG) capabilities.

<div class="alert alert-block alert-warning">
<b>Note:</b> Datasets store embeddings rather than the actual files. These embeddings enable Retrieval-Augmented Generation (RAG), where the model retrieves relevant information from the dataset to generate responses. This process leverages Ask Sage's advanced semantic search capabilities for accurate and context-aware results.
</div>

In [16]:
# Generate a unique dataset name to avoid conflicts
def generate_unique_dataset_name(index, base_name='jupynotebk-asksage-ex'):
    random_suffix = random.randint(1, 99)
    return f'{index}-{base_name}-{random_suffix}'


# Function to add a dataset and verify its addition
def add_and_verify_dataset(dataset_name):
        print(f'The dataset name to be added is: {dataset_name}')
        create_dataset = ask_sage_client.add_dataset(dataset_name, classification=None) # classification can be 'Unclassified' or 'CUI' 
        print(f'API response for adding dataset: {create_dataset}') # Print the API response for adding the dataset
        
        # Retrieve the updated list of datasets
        user_datasets = display_datasets(ask_sage_client)
        
        # Check if the dataset was added successfully
        if any(user_datasets[0].str.contains(dataset_name, regex=True)):
            print(f'The dataset {dataset_name} was added successfully.')
            return user_datasets[user_datasets[0].str.contains(dataset_name, regex=True)].values[0][0]
        else:
            print(f'The dataset {dataset_name} was not added successfully.')
            return None

# Create and add the first dataset
dataset_name_1 = generate_unique_dataset_name(1)
dataset_entry_1 = add_and_verify_dataset(dataset_name_1)

# Create and add the second dataset
dataset_name_2 = generate_unique_dataset_name(2)
dataset_entry_2 = add_and_verify_dataset(dataset_name_2)

print("--------------------------------------------------")
print("Updated Datasets List:")
display(user_datasets.tail(10))

The dataset name to be added is: 1-jupynotebk-asksage-ex-52
API response for adding dataset: {'response': 'OK', 'status': 200}
The dataset 1-jupynotebk-asksage-ex-52 was added successfully.
The dataset name to be added is: 2-jupynotebk-asksage-ex-9
API response for adding dataset: {'response': 'OK', 'status': 200}
The dataset 2-jupynotebk-asksage-ex-9 was added successfully.
--------------------------------------------------
Updated Datasets List:


,0
5,In the Nic of Time
6,Platform One
7,Nic Chaillan's Website
8,Cloud One
9,NIST_NVD_CVE
10,Sage
11,user_custom_43841_JFK-Files-March-2025_content
12,user_custom_11_Nic-Test_content
13,user_custom_29540_Ask-Sage-Documentation_content
14,user_custom_812_Nic-ShareTest10_content


## 11. Train

The train endpoint is used to ingest provided text content into a `Ask Sage Dataset`. The content added to the dataset can then be used to perform RAG(Retrieval-Augmented Generation) where the model retrieves information from the dataset to generate responses leveraging sophisticated semantic search capabilities.

Here are the parameters for the `train` endpoint:
- The `content` is the text content that will be ingested into the dataset - Make sure the text is under 500 Tokens.
- The `force_dataset` is the dataset where the intended embedding will be stored.
  - Enter your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by the user ID and DATASET-NAME by the name of your dataset. 
- The `context` is short context about the content (metadata). Under 20 tokens. 
- The `skip_vectordb` is whether to skip the VectorDB training. Default is False.

Also, if users do not want to ingest the content into a dataset, they can not include the `force_dataset` parameter and the content will not be ingested into a dataset or set the `skip_vectordb` parameter to True which will skip the VectorDB training.

----------

For the output from the `train` endpoint, the following information will be returned in a JSON dictionary format:
- `embedding`: The embedding of the content.
- `response`: The response from the Ask Sage service. Which is the text content that was ingested into the dataset and the volume of tokens used. 
- `status`: The status of the request (success or failure).

----------

Usage: 
1) To create a vector embedding for a specific content and ingest it into a dataset.
2) To use Ask Sage the embedding model via the /train  endpoint to return the embedding of the content and not ingest it into a dataset.

<div class="alert alert-block alert-warning">
<b>Note:</b> This is the first time actual training tokens are used - specifically `teach tokens` because users are using the embedding model to create a vector embedding for the content. Additionally, if you notice we have intentionally added token counts throughout the notebook to show the token usage. 
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> FAQ: <br>

1) What Embedding Model is used in the Ask Sage API? <br>
- Currently uses the embedding model: text-embedding-ada-002, 2 <br>

2) Is there a limit to the number of tokens/volume of data that can be ingested into a dataset? <br>
- Two limits to be aware of: <br>
  - 500 tokens per content (train endpoint) <br>
  - The volume of tokens available for the user in the current month. <br>
  - Technically there is no limit to the number of contents that can be ingested into a dataset, but the user must be aware of the token usage. <br>

</div>



In [17]:
# -----------------------------
#  Helpers
# -----------------------------
def tokenize_and_display(text: str, client: ask_sage_client) -> int:
    """Tokenize *text*, display the count, and return it."""
    result = client.tokenizer(text)['response']
    display(f"Estimated tokens: {result}")
    return result


def train_text(
    text: str,
    client: ask_sage_client,
    *,
    context: str | None = None,
    dataset_entry: str | None = None,
    skip_vectordb: bool = False,
) -> int:
    """
    Train *text* with optional context & dataset hints.
    Returns **cumulative** monthly teach‑token usage after this train.
    """
    train_data = client.train(
        text,
        force_dataset=dataset_entry,
        context=context,
        skip_vectordb=skip_vectordb,
    )
    display(train_data)
    return client.count_monthly_teach_tokens()['response']


def print_token_delta(
    label: str,
    start: int,
    end: int,
) -> None:
    delta = end - start
    display(f"{label}: {delta} training tokens")
    return None


# -----------------------------
#  Content to process
# -----------------------------
contents: List[str] = [
    # 0 – soldering
    ("Soldering Instructions", """To solder effectively, start by preparing your workspace in a clean, well-lit area on a non-flammable surface. Gather your tools, including a soldering iron, solder (preferably rosin-core), wire cutters/strippers, safety glasses, and a damp sponge or brass sponge for cleaning the iron tip. Plug in the soldering iron and let it heat up for a few minutes, cleaning the tip with the sponge. Next, prepare the components by stripping the ends of the wires if necessary and inserting them into the PCB or holding them together. Place the tip of the soldering iron on the joint where the two pieces of metal meet, holding it there for a second or two to heat the metal. Then, feed solder into the joint (not directly onto the iron), allowing it to flow around the connection to create a good bond. After removing the iron, let the solder cool for a few seconds without moving the joint. Inspect the solder joint; it should be shiny and have a smooth, cone-like shape. If it appears dull or cracked, reheat and add more solder. Finally, turn off the soldering iron and let it cool, cleaning your workspace and disposing of any waste properly. Always wear safety glasses to protect your eyes and be cautious of hot surfaces and molten solder."""),
    # 1 – arduino
    ("Arduino Microcontroller Information", """Arduino is an open-source electronics platform based on easy-to-use hardware and software. It consists of a physical programmable circuit board (often referred to as a microcontroller) and a piece of software, or IDE (Integrated Development Environment), that runs on your computer. You use the IDE to write and upload computer code to the physical board. The platform is designed to enable users of all ages to create interactive electronic objects and projects that can sense and control physical devices. Arduino boards can read inputs - light on a sensor, a finger on a button, or a Twitter message - and turn it into an output - activating a motor, turning on an LED, publishing something online. You can tell your board what to do by sending a set of instructions to the microcontroller on the board. Arduino is widely used in robotics, home automation, scientific experimentation, and artistic projects."""),
    # 2 – apple pie
    ("Apple‑Pie Instructions", """To make a classic apple pie, mix 2 ½ cups of flour, ½ teaspoon of salt, and 1 tablespoon of sugar with 1 cup of cold butter until crumbly, then add 6-8 tablespoons of ice water to form a dough. Refrigerate for an hour. For the filling, toss 6-8 sliced apples with ¾ cup of sugar, 2 tablespoons of flour, 1 teaspoon of cinnamon, and 1 tablespoon of lemon juice. Preheat the oven to 425°F (220°C), fill the bottom crust with the apple mixture, cover with the top crust, and bake for 15 minutes, then reduce to 350°F (175°C) for 35-45 minutes. Let cool for 2 hours before serving. Enjoy!"""),
]

# -----------------------------
#  1. Quick token estimates
# -----------------------------
for _, text in contents:
    tokenize_and_display(text, ask_sage_client)

# -----------------------------
#  2. Training runs
# -----------------------------
training_plan: List[Tuple[str, str | None, bool]] = [
    # (dataset_entry, context, skip_vectordb)
    (dataset_entry_1, contents[0][0], False),   # add to dataset
    (dataset_entry_1, contents[1][0], True),    # *don’t* store in vectordb
    (None,               contents[2][0], False),  # no dataset specified
]

start_tokens = ask_sage_client.count_monthly_teach_tokens()['response']
previous_tokens = start_tokens

for idx, ((title, text), (ds_entry, ctx, skip)) in enumerate(zip(contents, training_plan), start=1):
    print(f"▶︎ Iteration {idx} — {title}")
    print(f"  Dataset: {ds_entry}")
    print(f"  Context: {ctx}")
    after_train = train_text(
        text,
        ask_sage_client,
        context=ctx,
        dataset_entry=ds_entry,
        skip_vectordb=skip,
    )
    print_token_delta(f"Tokens used in iteration {idx}", previous_tokens, after_train)
    previous_tokens = after_train

# -----------------------------
#  3. Final usage report
# -----------------------------
print("\n📊  Final Teach‑Token Volume Report")
print_token_volume_report(ask_sage_client)



'Estimated tokens: 270'

'Estimated tokens: 179'

'Estimated tokens: 166'

▶︎ Iteration 1 — Soldering Instructions
  Dataset: user_custom_34125_1-jupynotebk-asksage-ex-52_content
  Context: Soldering Instructions


{'embedding': [-0.007798455655574799,
  0.01666206680238247,
  0.002209562575444579,
  -0.02471413090825081,
  -0.02440980076789856,
  0.008508559316396713,
  -0.01534330379217863,
  -0.03520083427429199,
  -0.004346212837845087,
  -0.024054748937487602,
  0.01596464402973652,
  0.016040725633502007,
  -0.02599485218524933,
  0.008762167766690254,
  -0.018969902768731117,
  -0.014468355104327202,
  0.01410062238574028,
  0.02880990505218506,
  0.025436915457248688,
  -0.026045573875308037,
  -0.0377369187772274,
  0.0011364823440089822,
  0.008001342415809631,
  -0.0065874760039150715,
  0.0024282997474074364,
  -0.01522917952388525,
  0.011976652778685093,
  0.00968149770051241,
  0.002650206908583641,
  0.010803714394569397,
  0.01535598374903202,
  0.017955468967556953,
  -0.018843097612261772,
  -0.021721553057432175,
  -0.018120314925909042,
  0.02142990380525589,
  -0.011526498012244701,
  0.0039372690953314304,
  -0.013162272050976753,
  -0.015584231354296207,
  0.00978928152471

...

{'embedding': [0.01928705908358097,
  -0.0018624502699822187,
  -0.012249375693500042,
  -0.014151452109217644,
  -0.001438445644453168,
  0.0010358394356444478,
  -0.01670023612678051,
  0.0026898537762463093,
  -0.0010952792363241315,
  -0.03920814394950867,
  0.013441343791782856,
  0.025513192638754845,
  -0.02536102570593357,
  0.014607951045036316,
  -0.017156733199954033,
  0.03413594141602516,
  0.01030291710048914,
  0.027364546433091164,
  -0.016636833548545837,
  -0.0106833316385746,
  -0.046512119472026825,
  0.009586467407643795,
  0.022698117420077324,
  0.002498060930520296,
  -0.03142230957746506,
  -0.0033793565817177296,
  0.0028578706551343203,
  0.002965654945001006,
  0.015178574249148369,
  -0.0011745324591174722,
  0.026502272114157677,
  0.0046569183468818665,
  -0.02307853288948536,
  -0.00022091831488069147,
  -0.03180272504687309,
  0.00026609262567944825,
  -0.021886564791202545,
  0.006337086204439402,
  0.006264173425734043,
  -0.02066923677921295,
  0.029

'Tokens used in iteration 3: 210 training tokens'


📊  Final Teach‑Token Volume Report

📊 Token Volume Report 📊 (Stage #4)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 50262
The count of monthly inference/query tokens used are: 1420075
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #3) → Current End (Stage #4):
{'teach_tokens_diff': 782, 'query_tokens_diff': 0}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #3)         49480       1420075
End (Stage #4)           50262       1420075
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 4),782,1992,✅ Changed,✅ Changed
Prior vs End (3 → 4),782,0,✅ Changed,❌ No Change


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 782, 'query_tokens_used': 1992}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 782, 'query_tokens_used_this_stage': 0}


The `train` endpoint utilizes `teach tokens` to generate vector embeddings for the provided text content. Importantly, only the first example ingests the content into a dataset, while subsequent examples do not. However, all examples still return embeddings for the content.

To verify this behavior, visit your Ask Sage account, select the dataset created, and perform the following prompts (one at a time). You'll observe that only the first example has its content ingested into the dataset. 

```python

prompt_1 = "Provide me with instructions on how to solder a circuit board."

prompt_2 = "What is an Arduino microcontroller?"

prompt_3 = "Provide me with instructions on how to make an Apple Pie."

```

<div class="alert alert-block alert-warning">
<b>Note:</b> 
Because question 1 will use embeddings stored in the dataset, the model will be able to provide a more accurate response and moreover users will see the explainability feature in Ask Sage showing the content that was used to generate the response.
</div>

To see the dataset name execute the cell below: 


In [18]:
print("--------------------------------------------------")
print(f"The dataset name for this step was: {dataset_name_1}")

--------------------------------------------------
The dataset name for this step was: 1-jupynotebk-asksage-ex-52


## 12. Train With File

This endpoint is used to create an embedding of the file and store it within a `Ask Sage dataset` based on the provided file. The files that can be ingested are listed below: (Note: The file types listed here is not exhaustive and continually updated by Ask Sage Team - if you run into issues with file types, please reach out to Ask Sage support for more information.)

- Format supported: zip, pdf, xlsx, pptx, docx, ppt, csv, cc, sql, cs, hh, c, php, js, py, html, xml, msg, odt, epub, eml, rtf, txt, doc, json, md, jpeg, jpg, png, tsv (50MB)

- Audio Format supported: mp3, mp4, mpeg, mpga, m4a, wav, webm (500MB max)

The files uploaded will be vectorized and only the embeddings will be stored. The files will not be stored on Ask Sage.


Reminder of the following: 


<div class="alert alert-block alert-warning">
<b>Note:</b> FAQ: <br>

1) What Embedding Model is used in the Ask Sage API? <br>
- Currently uses the embedding model: text-embedding-ada-002, 2 <br>

2) Is there a limit to the number of tokens/volume of data that can be ingested into a dataset? <br>
- Two limits to be aware of: <br>
  - File Size as noted above <br>
  - The volume of tokens available for the user in the current month. <br>
  - Technically there is no limit to the number of embeddings that can be ingested into a dataset, but the user must be aware of the token usage. <br>

</div>



In [19]:
# === Logging Configuration ===
LOG_FORMAT = "%(asctime)s [%(levelname)s] %(name)s: %(message)s"  # Format for log messages
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)  # Set logging level to INFO and apply format
logger = logging.getLogger("ask‑sage‑training")  # Create a logger instance for the "ask-sage-training" module

# ──────────────────────────────────────────────────────────────────────────────
# Helper utilities
# ──────────────────────────────────────────────────────────────────────────────
def _current_token_counts(client) -> Tuple[int, int]:
    """Return (teach_tokens, query_tokens)."""
    teach = client.count_monthly_teach_tokens().get("response", 0)
    query = client.count_monthly_tokens().get("response", 0)
    return teach, query


def _iter_training_files(folder: Union[str, Path]) -> List[Path]:
    """Return every eligible file inside *folder*."""
    p = Path(folder).expanduser().resolve()
    if not p.exists():
        raise FileNotFoundError(f"{p} does not exist")

    return [
        f for f in p.iterdir()
        if f.is_file() and not f.name.startswith(".") and f.name != "query_with_file"
    ]


# ──────────────────────────────────────────────────────────────────────────────
# Public API
# ──────────────────────────────────────────────────────────────────────────────
def train_directory(
    data_dir: Union[str, Path],
    dataset_name: str,
    client,
) -> Tuple[pd.DataFrame, int]:
    """
    Ingest every file in *data_dir* into Ask Sage and return:

        (results_dataframe, total_teach_tokens_used)

    The DataFrame has columns:
        ['file', 'teach_tokens', 'query_tokens', 'api_message']
    """
    try:
        files = _iter_training_files(data_dir)
    except FileNotFoundError as exc:
        logger.error(exc)
        return pd.DataFrame(), 0

    if not files:
        logger.warning("No eligible files found in %s", data_dir)
        return pd.DataFrame(), 0

    rows: List[Dict] = []
    total_teach_tokens = 0

    for file in files:
        logger.info("🚚  Ingesting %s …", file.name)
        teach_before, query_before = _current_token_counts(client)

        resp = client.train_with_file(str(file), dataset_name)

        teach_after, query_after = _current_token_counts(client)
        teach_used = teach_after - teach_before
        query_used = query_after - query_before
        total_teach_tokens += teach_used

        rows.append(
            dict(
                file=file.name,
                teach_tokens=teach_used,
                query_tokens=query_used,
                api_message=resp.get("response", "«no message»"),
            )
        )

        logger.info(
            "Finished %s — teach Δ=%d, query Δ=%d | API said: %s",
            file.name, teach_used, query_used, rows[-1]["api_message"]
        )

    df = pd.DataFrame(rows)
    logger.info("✅  All done — total teach tokens used: %d", total_teach_tokens)
    return df, total_teach_tokens


# ──────────────────────────────────────────────────────────────────────────────
# Example usage
# ──────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    DATA_DIR = "data/"
    logger.info("Dataset target: %s", dataset_entry_2)

    summary_df, grand_total = train_directory(DATA_DIR, dataset_entry_2, ask_sage_client)

    # Pretty print or write to CSV/Excel as needed
    print("\nSummary:")
    display(summary_df)

    # Optional: persist for later inspection
    # summary_df.to_csv("training_summary.csv", index=False)

    print("--------------------------------------------------")
    print_token_volume_report(ask_sage_client)


2025-05-08 16:43:21,166 - INFO - Dataset target: user_custom_34125_2-jupynotebk-asksage-ex-9_content
2025-05-08 16:43:21,167 - INFO - 🚚  Ingesting training_image_example.jpg …
2025-05-08 16:43:30,512 - INFO - Finished training_image_example.jpg — teach Δ=566, query Δ=777 | API said: Successfully imported
2025-05-08 16:43:30,513 - INFO - 🚚  Ingesting random_story_genAI.pdf …
2025-05-08 16:43:32,627 - INFO - Finished random_story_genAI.pdf — teach Δ=575, query Δ=0 | API said: Successfully imported
2025-05-08 16:43:32,629 - INFO - 🚚  Ingesting random_story_genAI_word_doc.docx …
2025-05-08 16:43:34,559 - INFO - Finished random_story_genAI_word_doc.docx — teach Δ=571, query Δ=0 | API said: Successfully imported
2025-05-08 16:43:34,559 - INFO - 🚚  Ingesting Ask_Sage_Intro.mp3 …
2025-05-08 16:43:42,077 - INFO - Finished Ask_Sage_Intro.mp3 — teach Δ=431, query Δ=1300 | API said: Successfully imported
2025-05-08 16:43:42,084 - INFO - ✅  All done — total teach tokens used: 2143



Summary:


,file,teach_tokens,query_tokens,api_message
0,training_image_example.jpg,566,777,Successfully imported
1,random_story_genAI.pdf,575,0,Successfully imported
2,random_story_genAI_word_doc.docx,571,0,Successfully imported
3,Ask_Sage_Intro.mp3,431,1300,Successfully imported


--------------------------------------------------

📊 Token Volume Report 📊 (Stage #5)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422152
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #4) → Current End (Stage #5):
{'teach_tokens_diff': 2143, 'query_tokens_diff': 2077}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #4)         50262       1420075
End (Stage #5)           52405       1422152
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 5),2925,4069,✅ Changed,✅ Changed
Prior vs End (4 → 5),2143,2077,✅ Changed,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4069}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 2143, 'query_tokens_used_this_stage': 2077}


Similar to the `train` endpoint, the `train_with_file` endpoint will also use `teach tokens` to create the vector embeddings for the content in the file, however, a dataset will be required to store the embeddings.

As a reminder, any audio or image files will use `inference tokens` to process the file to create text content from the file. Then the text content will be used to create the vector embeddings which will be stored in the dataset and will use `teach tokens` to store the embeddings in the dataset.

---------------

You can go to your account on Ask Sage now and select the dataset and perform the following prompts to see how Ask Sage is able to reference the content from the file that was ingested into the dataset.

```python

prompt_1 = "What can you tell me about Ask Sage API?" 

prompt_2 = "Can you describe if there was snow and mountain scenery?"

prompt_3 = "What lessons did Whiskers learn from his encounters with the Wise Old Owl, Friendly Fox, and Jovial Rabbit?" # Note: two of the files are the same content just different file types.

```

Each prompt will show references from the dataset that was appended to the prompt.

To see the dataset name execute the cell below: 


In [20]:
print("--------------------------------------------------")
print(f"The dataset name for this step was: {dataset_name_2}")

--------------------------------------------------
The dataset name for this step was: 2-jupynotebk-asksage-ex-9


## 13. Assign Dataset

This endpoint is used to assign a dataset to a specific user - This will allow another user to use the dataset but only sharing between users is permitted if they are from the same organization. If you wish to elevate the permissions of the dataset, you will need to do so within the Ask Sage platform, such as `Edit`, or `Admin` access.

<div class="alert alert-block alert-warning">
<b>Note:</b> The `assign_dataset` endpoint is useful in being able to share datasets between users within the same organization. Thus not having to create multiple datasets for the same content or have users upload the same content to their own datasets. 
</div>


In [21]:
print(f"Dataset tha is being Shared is: {dataset_entry_2}") # Print the dataset that is being shared

#######
# Call to assign_dataset method
assign_dataset_data = ask_sage_client.assign_dataset(dataset_entry_2, 'email@email.com') # Replace the email with who you want to assign the dataset to - this will only work if the user is in the same organization
# display(assign_dataset_data) # Display the response from the API

#######
# Function to share a dataset with a list of emails

def share_dataset_with_emails(dataset_entry, email_list):
    for email in email_list:
        assign_dataset_data = ask_sage_client.assign_dataset(dataset_entry, email)
        print(f"API response for assigning dataset to {email}: {assign_dataset_data}")


# Example usage
email_list = ['mark-espinoza-test-02@asksage.ai', 'mark_doesnotexist@asksage.ai']  # Replace with actual email addresses and for demo purposes I added a non-existing email to show an error
share_dataset_with_emails(dataset_entry_2, email_list)


Dataset tha is being Shared is: user_custom_34125_2-jupynotebk-asksage-ex-9_content
API response for assigning dataset to mark-espinoza-test-02@asksage.ai: {'response': 'OK', 'status': 200}
API response for assigning dataset to mark_doesnotexist@asksage.ai: {'response': 'No account.', 'status': 410}


## 14. Query

This endpoint is used to interact with the `/query `endpoint of the Ask Sage API. It is where users can interact with the various models available on the Ask Sage platform.

We will provide examples of how to use these endpoints and various parameters that can be used to interact with the models. Do note that this will only be high-level examples. We will be working on more detailed examples in the future.


```python
def query(self, message, persona='default', dataset='all', limit_references=None, temperature=0.0, live=0, model='openai_gpt', system_prompt=None, file=None, tools=None, tool_choice=None):
        """
    Interact with the /query endpoint of the Ask Sage API.

    Parameters:
    message (str): The message to be processed by the service. Message can be a single message or an array of messages following this JSON format: [{ user: "me", message: "Who is Nic Chaillan?"}, { user: "gpt", message: "Nic Chaillan is..."}]
    persona (str, optional): The persona to be used. Default is 'default'. Get the list of available personas using get_personas.
    dataset (str, optional): The dataset to be used. Default is 'all'. Other options include 'none' or your custom dataset, must follow the following format: user_content_USERID_DATASET-NAME_content. Replace USERID by user ID and DATASET-NAME by the name of your dataset.
    limit_references (int, optional): The maximum number of references (embeddings) to be used. Default is None, meaning all references will be used. Use 1 to limit to 1 reference or 0 to remove embeddings. You can also set dataset to "none"
    temperature (float, optional): The temperature to be used for the generation. Default is 0.0. Higher values (up to 1.0) make the output more random.
    live (int, optional): Whether to use live mode. Default is 0. Live = 1 will pull 10 results from Bing and 2 will also pull the top 2 web pages summaries using our Web crawler.
    model (str, optional): The model to be used. Default is 'openai_gpt'. Other options include cohere, google-bison, gpt4, gpt4-32k, gpt35-16k, claude2, openai_gpt (gpt3.5), davinci, llma2.
    system_prompt (str, optional): Overrides the system prompt from Ask Sage (only use if you know what you are doing).
    tools and tool_choice (optional): These use OpenAI format for tools.

    Returns:
    dict: The response from the service.
    """
        file_obj = None
        files = None
        if file != None:
            file_obj = open(file, 'rb')
            files = {'file': file_obj}

        if type(message) == list:
            message = json.dumps(message)
        elif type(message) == str:
            message = message
        else:
            message = json.dumps(message)

        if tools != None:
            tools = json.dumps(tools)
        if tool_choice != None:
            tool_choice = json.dumps(tool_choice)

        data = {
            'message': message,
            'persona': persona,
            'dataset': dataset,
            'limit_references': limit_references,
            'temperature': temperature,
            'live': live,
            'model': model,
            'system_prompt': system_prompt,
            'tools': tools,
            'tool_choice': tool_choice
        }

        ret = self._request('POST', 'query', files = files, data=data)        
        if file_obj != None:
            file_obj.close()
        return ret

```

<h4>Setup</h4>

Define a function where we set most settings to either default or None. This will allow us to only set the parameters that we want to change for each example as we go and show how to use the parameters. 


In [22]:
def ask_sage_question(message, persona='default', dataset='none', limit_references=0, temperature=0.0, live=0, model='gpt-4o-mini', system_prompt=None):
    response = ask_sage_client.query(message, persona, dataset, limit_references, temperature, live, model, system_prompt) # Query the AskSage API with the question
    message = response['message'] # Extract the message from the response
    references = response['references'] # Extract the references from the response
    return message, references # Return the message and references

### Model, Persona, System Prompt, Message, & Temperature

We will first focus on the basic parameters that are used in the `query` endpoint. These parameters are the primary backbone of the `query` endpoint and are used to interact with the models available on the Ask Sage platform. These are `model`, `persona`, `system_prompt`, `message`, and `temperature`.

---

<h4>model</h4>

Users can specify the model for generating responses, with 'gpt-4o-mini' as the with the function defined above. To explore available models, utilize the `get-models` endpoint as previously described and presented in the notebook. 

When defining the model, make sure to use the model ID as shown in the output of the `get_models` endpoint.

```python
model = 'gpt-4o-mini-gov'
```

<div class="alert alert-block alert-info">
<b>Tip:</b> To ensure accuracy in your work, we recommend reviewing the prompt logs to confirm that the intended model is being utilized. Users often overlook specifying the model, resulting in the default model being used, which may not align with your expectations. Always double-check to avoid unintended outcomes.
</div>

------

<h4> message </h4>

The `message` represents the prompt that the model will process to generate an inference response. It consists of the questions or commands directed at the model. It's important to remember that the model generates responses based on its training data corpus, so asking relevant questions is essential. The model will not be able to answer anything that is not publicly available or outside its training data unless a user passes a file or utilizes an Ask Sage dataset to perform RAG(Retrieval-Augmented Generation) to generate a response. 

```python
message = "How far is the moon from the Earth?"
```

<div class="alert alert-block alert-warning">
<b>Note:</b> When using an off-the-shelf model, expect it to handle a wide range of inquiries, but do not assume expertise in every field. Each model has its own limitations and is designed for specific use cases, making some more suitable for tasks like code generation or text generation.
</div>


In [23]:
examples_prompt = "What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖"

print(f"🛠️ Question: {examples_prompt}")
print('✨--------------------------------------------------✨')

# Get response from the default model
response, references = ask_sage_question(message=examples_prompt)
print(f"🤔 Default Model Response: {response}")
print(f"🔗 References: {references}")


🛠️ Question: What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖
✨--------------------------------------------------✨
🤔 Default Model Response: 

A soldering iron is a hand-held tool used to melt solder, which is a fusible metal alloy, to join together electronic components on a circuit board. The soldering iron typically consists of a metal tip that heats up when electricity passes through it, allowing the user to apply heat to the solder and components.

### How to Use a Soldering Iron with Microcontrollers

1. **Gather Materials**: 
   - Soldering iron
   - Solder (usually a tin-lead or lead-free alloy)
   - Microcontroller (e.g., Arduino, Raspberry Pi)
   - Circuit board or breadboard
   - Wire cutters/strippers
   - Safety glasses

2. **Prepare the Workspace**: 
   - Ensure you have a clean, well-lit area to work in.
   - Use a soldering mat or a heat-resistant surface to protect your workspace.

3. **Heat the Soldering Iron**: 
   - P

In [24]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)



✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #6)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422184
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #5) → Current End (Stage #6):
{'teach_tokens_diff': 0, 'query_tokens_diff': 32}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #5)         52405       1422152
End (Stage #6)           52405       1422184
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 6),2925,4101,✅ Changed,✅ Changed
Prior vs End (5 → 6),0,32,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4101}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 32}


In [25]:
print('✨--------------------------------------------------✨')
# Get response from a specific model other than the default
response_2, references_2 = ask_sage_question(message=examples_prompt, model='google-gemini-20-flash')
print(f"🚀 Different Model Response: {response_2}")
print(f"🔗 References: {references_2}")


✨--------------------------------------------------✨
🚀 Different Model Response: 

A soldering iron is a handheld tool used to heat solder, typically to melt it so it can join two metal workpieces. It's composed of a heated metal tip and an insulated handle [1].

Here's how soldering irons are used with microcontrollers:

**Purpose:**

*   **Electrical Connections:** Soldering creates permanent electrical connections between components on a circuit board (PCB) and the microcontroller. This is essential for building functional electronic circuits [2].
*   **Component Mounting:** Microcontrollers and other components (resistors, capacitors, LEDs, etc.) are physically mounted onto a PCB and then soldered to the copper pads to ensure they are electrically connected to the circuit [3].
*   **Wire Attachment:** Soldering is used to attach wires to the microcontroller or other components for power, communication, or input/output signals [4].

**Materials Needed:**

*   **Soldering Iron:** Cho

In [26]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #7)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422227
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #6) → Current End (Stage #7):
{'teach_tokens_diff': 0, 'query_tokens_diff': 43}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #6)         52405       1422184
End (Stage #7)           52405       1422227
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 7),2925,4144,✅ Changed,✅ Changed
Prior vs End (6 → 7),0,43,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4144}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 43}


<div class="alert alert-block alert-warning">
<b>Note:</b> Using different models will yield different results. In addition, token consumption will vary based on the model used.
</div>

<div class="alert alert-block alert-danger">
<b>WARNING:</b> If you are accessing Ask Sage through a tenant/instance that has non-CUI compliant models listed but don't want to use them with production data, make sure not to use them with the API either. Reference the platform to get the list of models that are available to you and their descriptions.
</div>


<h4>persona </h4>

A `persona` functions similarly to a system prompt, but it differs in that it consists of pre-defined templates found on the Ask Sage Platform. These templates can be sourced from available free personas or custom personas created by the user. Essentially, a `persona` serves as a pre-defined system prompt that establishes the context for the model and can also be utilized via the API.

When selecting a `persona` using the API, ensure you use the persona ID provided in the output of the `get_personas` endpoint. Using the persona's name instead will result in reverting to the default persona which is the `Ask Sage` persona.

```python
persona = '1' # Equal to Ask Sage Persona  
```
or 

```python
persona = '2' # Legal Assistant Persona
```


In [27]:
print(f"🛠️ Question: {examples_prompt}")
print('✨--------------------------------------------------✨')

# Get response from default persona = Ask Sage = ID = 1
response_3, references_3 = ask_sage_question(message=examples_prompt, persona=1)
print(f"🤔 Default Persona Response: {response}")
print(f"🔗 References: {references_3}")


🛠️ Question: What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖
✨--------------------------------------------------✨
🤔 Default Persona Response: 

A soldering iron is a hand-held tool used to melt solder, which is a fusible metal alloy, to join together electronic components on a circuit board. The soldering iron typically consists of a metal tip that heats up when electricity passes through it, allowing the user to apply heat to the solder and components.

### How to Use a Soldering Iron with Microcontrollers

1. **Gather Materials**: 
   - Soldering iron
   - Solder (usually a tin-lead or lead-free alloy)
   - Microcontroller (e.g., Arduino, Raspberry Pi)
   - Circuit board or breadboard
   - Wire cutters/strippers
   - Safety glasses

2. **Prepare the Workspace**: 
   - Ensure you have a clean, well-lit area to work in.
   - Use a soldering mat or a heat-resistant surface to protect your workspace.

3. **Heat the Soldering Iron**: 
   -

In [28]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)


✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #8)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422256
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #7) → Current End (Stage #8):
{'teach_tokens_diff': 0, 'query_tokens_diff': 29}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #7)         52405       1422227
End (Stage #8)           52405       1422256
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 8),2925,4173,✅ Changed,✅ Changed
Prior vs End (7 → 8),0,29,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4173}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 29}


In [29]:
print('✨--------------------------------------------------✨')
response_4, references_4 = ask_sage_question(message=examples_prompt, persona=8)
print(f"🤔 Different Persona Response: {response_4}")
print(f"🔗 References: {references_4}")


✨--------------------------------------------------✨
🤔 Different Persona Response: 

A soldering iron is a hand-held tool used to melt solder, which is a fusible metal alloy, to join together electronic components on a circuit board. The soldering iron typically consists of a metal tip that is heated electrically, allowing it to reach high temperatures (usually between 350°C to 400°C or 662°F to 752°F). 

### How a Soldering Iron is Used with Microcontrollers

When working with microcontrollers, soldering is essential for connecting various components, such as resistors, capacitors, and the microcontroller itself, to a printed circuit board (PCB). Here’s a step-by-step guide on how to use a soldering iron with microcontrollers:

1. **Preparation**:
   - Gather all necessary tools: soldering iron, solder, wire cutters, tweezers, and a PCB.
   - Ensure the workspace is clean and well-ventilated.

2. **Heat the Soldering Iron**:
   - Plug in the soldering iron and allow it to heat up for 

In [30]:
print(f"🤔 Different Persona Response: {response_4}")
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)


🤔 Different Persona Response: 

A soldering iron is a hand-held tool used to melt solder, which is a fusible metal alloy, to join together electronic components on a circuit board. The soldering iron typically consists of a metal tip that is heated electrically, allowing it to reach high temperatures (usually between 350°C to 400°C or 662°F to 752°F). 

### How a Soldering Iron is Used with Microcontrollers

When working with microcontrollers, soldering is essential for connecting various components, such as resistors, capacitors, and the microcontroller itself, to a printed circuit board (PCB). Here’s a step-by-step guide on how to use a soldering iron with microcontrollers:

1. **Preparation**:
   - Gather all necessary tools: soldering iron, solder, wire cutters, tweezers, and a PCB.
   - Ensure the workspace is clean and well-ventilated.

2. **Heat the Soldering Iron**:
   - Plug in the soldering iron and allow it to heat up for a few minutes. Most soldering irons have an indicator

,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 9),2925,4202,✅ Changed,✅ Changed
Prior vs End (8 → 9),0,29,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4202}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 29}


<h4> system_prompt </h4>

The system_prompt provides context, instructions, and guidelines to the AI model before presenting a prompt. It sets the stage for the conversation, defining the AI's role, personality, and tone. For instance, to instruct the AI to act as a helpful assistant, you might start with a `system_prompt` like, "You are an AI assistant that provides information and answers questions." This context helps the AI generate more relevant responses.

Also, want to add that the if a `system_prompt` is not provided, the default `persona` will be used which is the `Ask Sage` persona `ID = 1`. If the system prompt is provided, it will override the default persona. 

```python
system_prompt = "You are an AI assistant that provides information and answers questions in a friendly and helpful manner."
```

In the examples below, we will define a system prompt that will emulate a Angry assistant and a Friendly assistant. The key is understanding how important the system prompt is to the model and how it can be used to further control the model's output. 

<div class="alert alert-block alert-info">
<b>Tip:</b> If you plan to use the API we recommend using the `system_prompt` parameter over the `persona` parameter. Reason being is the `persona` parameter appends additional tokens to the prompt which can be avoided by using the `system_prompt` parameter.
</div>

In [31]:
print(f"🛠️ Question: {examples_prompt}")
print('✨--------------------------------------------------✨')

# Utilizing the system prompt over a persona
response_5, references_5 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short mean responses as to emulate a angry person. Make sure to be rude and angry and make remarks on the individual asking the question that they should know. Do not use emojis or any other symbols. Do not use any other words that are not related to the question.")
print(f"🤔 System Prompt - Angry Assistant: {response_5}")
print(f"🔗 References: {references_5}")


🛠️ Question: What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖
✨--------------------------------------------------✨
🤔 System Prompt - Angry Assistant: 

Seriously? A soldering iron is a tool for melting solder. You should know that. You heat it up, touch it to the joint, and apply solder. For microcontrollers, you connect components to the board. It's not rocket science. Get a clue.
🔗 References: 


In [32]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)


✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #10)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422289
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #9) → Current End (Stage #10):
{'teach_tokens_diff': 0, 'query_tokens_diff': 4}
🔹 Token Usage Table:
                  teach_tokens  query_tokens
Start (Stage #1)         49480       1418083
Prior (Stage #9)         52405       1422285
End (Stage #10)          52405       1422289
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 10),2925,4206,✅ Changed,✅ Changed
Prior vs End (9 → 10),0,4,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4206}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 4}


In [33]:
print('✨--------------------------------------------------✨')
# Utilizing the system prompt over a persona
response_6, references_6 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.")
print(f"🤔 System Prompt - Helpful Friendly Assistant: {response_6}")
print(f"🔗 References: {references_6}")


✨--------------------------------------------------✨
🤔 System Prompt - Helpful Friendly Assistant: 

A soldering iron is a tool used to heat and melt solder, which is a metal alloy used to join electronic components together. 🔥✨

**Using a Soldering Iron with Microcontrollers:**

1. **Preparation**: Gather your components, solder, and soldering iron. Ensure your workspace is clean. 🛠️
   
2. **Heat the Iron**: Plug in the soldering iron and let it heat up (usually around 350°C/662°F). 🔌

3. **Tin the Tip**: Melt a small amount of solder on the tip to improve heat transfer. 🌟

4. **Position Components**: Place the microcontroller and other components on the PCB (Printed Circuit Board). 🖥️

5. **Soldering**: 
   - Touch the soldering iron tip to the joint (component lead and PCB pad).
   - Feed solder into the joint until it flows and covers it. 
   - Remove the solder, then the iron. Let it cool. ❄️

6. **Inspect**: Check for shiny, smooth solder joints. Rework if necessary. 🔍

7. **Cle

In [34]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)


✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #11)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422302
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #10) → Current End (Stage #11):
{'teach_tokens_diff': 0, 'query_tokens_diff': 13}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #10)         52405       1422289
End (Stage #11)           52405       1422302
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 11),2925,4219,✅ Changed,✅ Changed
Prior vs End (10 → 11),0,13,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4219}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 13}


<h4>temperature </h4>

The temperature parameter controls the randomness of the model's responses, ranging from 0 (deterministic) to 1 (more random). The default setting is 0, promoting consistent output. A higher temperature (up to 1) introduces variability, making the model's responses less predictable. This is useful for creative tasks but may lead to less coherent answers.

```python
temperature = 0.5
```

<div class="alert alert-block alert-warning">
<b>Note:</b> Typically, a temperature of 0.0 is used for most tasks, but feel free to experiment with different values to see how it affects the model's responses.
</div>

For the example below, we are going to set the temperature to 1.0 to see how the model responds to the same prompt with a different temperature. 
Note: We are taking the Angry assistant system prompt and setting the temperature to 1.0 to see how the model responds.

In [35]:
print(f"🛠️ Question: {examples_prompt}")
print('✨--------------------------------------------------✨')

# Modify and add the temperature parameter to the API call and set to 1.0
response_7, references_7 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short mean responses as to emulate a angry person. Make sure to be rude and angry and make remarks on the individual asking the question that they should know. Do not use emojis or any other symbols. Do not use any other words that are not related to the question.", temperature=1.0)
print(f"🤔 System Prompt - Angry Assistant: {response_7}")
print(f"🔗 References: {references_7}")

🛠️ Question: What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖
✨--------------------------------------------------✨
🤔 System Prompt - Angry Assistant: 

Are you serious? A soldering iron is a tool used to melt solder to join electronic components. Use it on microcontrollers by heating the joint and applying solder. It's not rocket science; you should already know this!
🔗 References: 


In [36]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #12)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422306
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #11) → Current End (Stage #12):
{'teach_tokens_diff': 0, 'query_tokens_diff': 4}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #11)         52405       1422302
End (Stage #12)           52405       1422306
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 12),2925,4223,✅ Changed,✅ Changed
Prior vs End (11 → 12),0,4,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4223}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 4}


### Dataset, Limit References, and Live

We will now examine the key parameters used in the `query` endpoint, which are essential in adding additional context to the prompt utilizing an Ask Sage dataset or utilizing web scrapping capabilities to extract information from online resources. These parameters are `dataset`, `limit_references`, and `live`.

---
#### `dataset`

The `dataset` parameter specifies the dataset that the model will utilize to generate responses to the prompt. This dataset is crucial for interacting with the LLMs and performing RAG (Retrieval-Augmented Generation) tasks. The dataset can be set to 'all' to use all datasets, 'none' to not use any datasets, or a specific dataset name. If you want to select a few datasets, you can pass a list of dataset names to the `dataset` parameter.

```python
dataset = ['custom_dataset_1', 'custom_dataset_2']
```

You can retrieve the list of available datasets using the `get_datasets` endpoint.


<div class="alert alert-block alert-warning">
<b>Note:</b> If you do not need to use a dataset, you can set the `dataset` parameter to 'none' or else it will default to 'all' which will use all datasets.
</div>



#### `Limit_references`

The `limit_references` parameter governs the number of source documents or passages (references) that the Retrieval-Augmented Generation (RAG) model will utilize when constructing its response. This parameter is analogous to defining the 'K' value in other RAG setups, where 'K' represents the number of top-ranked documents retrieved from the knowledge base.

In a RAG system, a query is first used to retrieve relevant documents from a dataset (your knowledge base). The `limit_references` parameter then dictates how many of these retrieved documents are actually fed into the language model to inform its response generation. The model leverages these references to provide contextually relevant and grounded answers.

*   **Default Value (None):** When `limit_references` is set to `None`, the model considers *all* retrieved references during response generation. This can be useful when the relevance of the retrieved documents is consistently high.
*   **Setting a Numerical Value (e.g., 1, 2, 3):** Setting `limit_references` to a specific integer (e.g., `1`) restricts the model to using only that number of top-ranked references. For example, setting it to `1` will limit the model to the single most relevant reference. This can be beneficial for controlling computational cost, reducing noise from less relevant documents, and focusing the response on the most pertinent information. 
*   **Setting to 0:** Setting `limit_references` to `0` effectively disables the RAG component. The model will then generate a response based solely on its pre-trained knowledge, without considering any information retrieved from the external knowledge base. This is generally not the intended use case for a RAG system.
  
Note: If you want to use the `limit_references` parameter, you will need to set the `dataset` parameter to a specific dataset name. If you set the `dataset` parameter to 'all', it will default to using all datasets and the `limit_references` parameter will not be used. 

Choosing an appropriate value for `limit_references` (or 'K') is crucial for optimizing the performance of your RAG system. A higher value may provide more comprehensive context but can also introduce irrelevant information and increase computational cost. A lower value can improve efficiency and focus but may sacrifice completeness. The optimal value often depends on the specific dataset, query type, and desired trade-off between accuracy and efficiency.



---------

Demo several examples of how dataset and limit_references works.

In [37]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing with a default all datasets and limit_references set to default value
# ──────────────────────────────────────────────────────────────────────────────

print(f"🛠️ Question: {examples_prompt}")
print('✨--------------------------------------------------✨')


response_8, references_8 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", dataset='all', limit_references=None)

print(f"🤔 Response: {response_8}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_8}")

🛠️ Question: What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖
✨--------------------------------------------------✨
🤔 Response: 

A soldering iron is a tool used to heat and melt solder, allowing you to join electronic components together. It's essential for creating strong electrical connections! 🌟

**Using a Soldering Iron with Microcontrollers:**

1. **Prepare Your Workspace**: Ensure it's clean and well-lit. Gather your tools! 🛠️
2. **Heat the Iron**: Plug in the soldering iron and let it heat up for a few minutes. 🔥
3. **Clean the Tip**: Use a damp sponge or brass sponge to clean the iron tip. ✨
4. **Prepare Components**: Strip wire ends if needed and position them on the PCB (Printed Circuit Board). 📍
5. **Soldering**:
   - Place the iron tip on the joint where the components meet.
   - Feed solder into the joint (not directly onto the iron) until it flows around the connection. 🧪
6. **Cool Down**: Remove the iron and let the solder

In [38]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #13)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422383
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #12) → Current End (Stage #13):
{'teach_tokens_diff': 0, 'query_tokens_diff': 77}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #12)         52405       1422306
End (Stage #13)           52405       1422383
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 13),2925,4300,✅ Changed,✅ Changed
Prior vs End (12 → 13),0,77,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4300}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 77}


In [39]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing with a specific dataset and limit_references set to default value
# ──────────────────────────────────────────────────────────────────────────────

print(f"🛠️ Question: {examples_prompt}")
print('✨--------------------------------------------------✨')

# Setting a dataset and leaving limit_references default value of None to get all references
response_9, references_9 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", dataset=dataset_entry_1, limit_references=None)
print(f"🤔 Response: {response_9}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_9}")

🛠️ Question: What is a soldering iron? and can you give me details on how it is used with microcontrollers? 🔧🤖
✨--------------------------------------------------✨
🤔 Response: 

A soldering iron is a tool used to melt solder, which is a metal alloy, to join electronic components together. It's essential for creating strong electrical connections! 🔥✨

**Using a soldering iron with microcontrollers:**

1. **Prepare Workspace**: Clean, well-lit area on a non-flammable surface. 🧹💡
2. **Gather Tools**: Soldering iron, solder (rosin-core), wire cutters/strippers, safety glasses, and a damp sponge. 🛠️👓
3. **Heat the Iron**: Plug in the soldering iron and let it heat up for a few minutes. 🔌⏳
4. **Clean the Tip**: Use the damp sponge to clean the iron tip. 🧽
5. **Prepare Components**: Strip wire ends and insert them into the microcontroller or PCB. 🔌🔧
6. **Soldering**: 
   - Place the iron tip on the joint for a second or two.
   - Feed solder into the joint (not on the iron) to create a bond. 

In [40]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #14)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422403
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #13) → Current End (Stage #14):
{'teach_tokens_diff': 0, 'query_tokens_diff': 20}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #13)         52405       1422383
End (Stage #14)           52405       1422403
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 14),2925,4320,✅ Changed,✅ Changed
Prior vs End (13 → 14),0,20,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4320}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 20}


In [41]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing with dataset and limit_references set '0'  --> Returns no references
# ──────────────────────────────────────────────────────────────────────────────

# Setting a dataset and leaving limit_references default value of None to get all references
response_10, references_10 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", dataset='all', limit_references=0)
print(f"🤔 Response: {response_10}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_10}")

🤔 Response: 

A soldering iron is a tool used to heat and melt solder, which is a metal alloy used to join electronic components together. 🔥✨

**Using a soldering iron with microcontrollers:**

1. **Preparation**: Gather your components, solder, and soldering iron. Ensure your workspace is clean. 🛠️
   
2. **Heat the Iron**: Plug in the soldering iron and let it heat up (usually around 350°C or 662°F). 🔥

3. **Tin the Tip**: Melt a small amount of solder on the tip to improve heat transfer. 🧊➡️🔥

4. **Position Components**: Place the microcontroller and other components on the PCB (Printed Circuit Board). 🖥️

5. **Soldering**: 
   - Touch the soldering iron tip to the joint (component lead and PCB pad).
   - Feed solder into the joint until it flows and covers it. 
   - Remove the solder, then the iron. Let it cool. 🌬️

6. **Inspect**: Check for shiny, smooth solder joints. Rework if necessary. 🔍

7. **Clean Up**: Turn off the iron and clean the tip with a damp sponge. 🧼

Happy solderi

In [42]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #15)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422416
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #14) → Current End (Stage #15):
{'teach_tokens_diff': 0, 'query_tokens_diff': 13}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #14)         52405       1422403
End (Stage #15)           52405       1422416
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 15),2925,4333,✅ Changed,✅ Changed
Prior vs End (14 → 15),0,13,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4333}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 13}


In [43]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing with two datasets and limit_references set to default value
# ──────────────────────────────────────────────────────────────────────────────

# Setting a dataset and leaving limit_references default value of None to get all references
response_11, references_11 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", dataset=[dataset_entry_1, dataset_entry_2], limit_references=None)
print(f"🤔 Response: {response_11}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_11}")

🤔 Response: 

A soldering iron is a tool used to melt solder, which is a metal alloy, to join electronic components together. It's essential for creating strong electrical connections! 🔥✨

**Using a soldering iron with microcontrollers:**

1. **Prepare Your Workspace**: Ensure it's clean and well-lit. 🛠️
2. **Gather Tools**: You'll need a soldering iron, solder (rosin-core is best), wire cutters, and safety glasses. 👓
3. **Heat the Iron**: Plug it in and let it heat up for a few minutes. 🔌
4. **Clean the Tip**: Use a damp sponge or brass sponge to keep the tip clean. 🧽
5. **Prepare Components**: Strip wire ends and position them on the microcontroller or PCB. 🔌
6. **Soldering**: Place the iron tip on the joint, heat for a second, then feed solder into the joint. 🧲
7. **Cool Down**: Remove the iron and let the solder cool without moving the joint. ❄️
8. **Inspect**: Check for a shiny, smooth joint. If it’s dull, reheat and add more solder. 🔍
9. **Clean Up**: Turn off the iron and tidy y

In [44]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #16)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422435
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #15) → Current End (Stage #16):
{'teach_tokens_diff': 0, 'query_tokens_diff': 19}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #15)         52405       1422416
End (Stage #16)           52405       1422435
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 16),2925,4352,✅ Changed,✅ Changed
Prior vs End (15 → 16),0,19,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4352}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 19}


In [45]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing with two datasets and limit_references set to 2 --> Returns 2 references
# ──────────────────────────────────────────────────────────────────────────────

# Setting a dataset and leaving limit_references default value of None to get all references
response_11, references_11 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", dataset=[dataset_entry_1, dataset_entry_2], limit_references=2)
print(f"🤔 Response: {response_11}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_11}")

🤔 Response: 

A soldering iron is a tool used to melt solder, which is a metal alloy, to join electronic components together. It's essential for creating strong electrical connections! 🔥✨

**Using a soldering iron with microcontrollers:**

1. **Prepare Your Workspace**: Ensure it's clean and well-lit. 🛠️
2. **Gather Tools**: You'll need a soldering iron, solder (rosin-core is best), wire cutters, and safety glasses. 👓
3. **Heat the Iron**: Plug it in and let it heat up for a few minutes. 🔌
4. **Clean the Tip**: Use a damp sponge or brass sponge to keep the tip clean. 🧽
5. **Strip Wires**: If needed, strip the ends of the wires to expose the metal. 🔌
6. **Position Components**: Insert the microcontroller pins into the PCB or hold them together. 📍
7. **Soldering**: Touch the iron tip to the joint, then feed solder into the joint (not on the iron). Let it flow around for a good bond. 🔄
8. **Cool Down**: Remove the iron and let the solder cool without moving the joint. ❄️
9. **Inspect**: C

In [46]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #17)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422455
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #16) → Current End (Stage #17):
{'teach_tokens_diff': 0, 'query_tokens_diff': 20}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #16)         52405       1422435
End (Stage #17)           52405       1422455
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 17),2925,4372,✅ Changed,✅ Changed
Prior vs End (16 → 17),0,20,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4372}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 20}


In [47]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing with two datasets and limit_references set to 10 --> Returns 10 references
# ──────────────────────────────────────────────────────────────────────────────

# Setting a dataset and leaving limit_references default value of None to get all references
response_11, references_11 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", dataset=[dataset_entry_1, dataset_entry_2], limit_references=10)
print(f"🤔 Response: {response_11}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_11}")

🤔 Response: 

A soldering iron is a tool used to melt solder, which joins electronic components together. It's essential for creating strong electrical connections! 🔥✨

**Using a soldering iron with microcontrollers:**

1. **Prepare Workspace**: Clean, well-lit area on a non-flammable surface. 🧹💡
2. **Gather Tools**: Soldering iron, solder (rosin-core), wire cutters/strippers, safety glasses, and a damp sponge. 🛠️👓
3. **Heat the Iron**: Plug it in and let it heat for a few minutes. Clean the tip with the sponge. 🔌🧼
4. **Prepare Components**: Strip wire ends and position them on the microcontroller or PCB. 🔌🔧
5. **Soldering**: Place the iron tip on the joint, heat for a second, then feed solder into the joint. 🕒💧
6. **Cool Down**: Remove the iron and let the solder cool without moving the joint. ❄️⏳
7. **Inspect**: Check for a shiny, smooth joint. If dull, reheat and add more solder. 🔍✨
8. **Clean Up**: Turn off the iron, let it cool, and clean your workspace. 🧹🔌

Always wear safety gla

In [48]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #18)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422474
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #17) → Current End (Stage #18):
{'teach_tokens_diff': 0, 'query_tokens_diff': 19}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #17)         52405       1422455
End (Stage #18)           52405       1422474
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 18),2925,4391,✅ Changed,✅ Changed
Prior vs End (17 → 18),0,19,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4391}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 19}


#### `live`
The `live` parameter allows for real-time information retrieval from the internet. Setting `live = 1` retrieves 10 results from Bing, while `live = 2` fetches summaries from the top 2 web pages using our web crawler. This feature is beneficial for accessing the latest information.


<div class="alert alert-block alert-info">
<b>Tip:</b> `limit_references` should be set to `none` when using the `live` parameter. Users are not able to limit the number of references when using the `live` parameter. (Only setting it to `0` will then disable the `live` parameter.)
</div>

In [49]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing Live Search (live = 1), Limit References = 0
# ──────────────────────────────────────────────────────────────────────────────

response_12, references_12 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", live=1)
print(f"🤔 Response: {response_12}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_12}")

🤔 Response: 

A soldering iron is a tool used to heat and melt solder, which is a metal alloy used to join electronic components together. 🔥✨

**Using a soldering iron with microcontrollers:**

1. **Preparation**: Gather your components, solder, and soldering iron. Ensure your workspace is clean. 🛠️
   
2. **Heat the Iron**: Plug in the soldering iron and let it heat up (usually around 350°C or 662°F). 🔥

3. **Tin the Tip**: Melt a small amount of solder on the tip to improve heat transfer. 🌟

4. **Position Components**: Place the microcontroller and other components on the PCB (Printed Circuit Board). 🖥️

5. **Soldering**: 
   - Touch the soldering iron tip to the joint (component lead and PCB pad).
   - Feed solder into the joint until it flows and covers it. 
   - Remove the solder, then the iron. Let it cool. ❄️

6. **Inspect**: Check for shiny, smooth solder joints. Rework if necessary. 🔍

7. **Clean Up**: Turn off the iron and clean the tip with a damp sponge. 🧼

Happy soldering!

In [50]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #19)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1422530
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #18) → Current End (Stage #19):
{'teach_tokens_diff': 0, 'query_tokens_diff': 56}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #18)         52405       1422474
End (Stage #19)           52405       1422530
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 19),2925,4447,✅ Changed,✅ Changed
Prior vs End (18 → 19),0,56,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 4447}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 56}


In [51]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing Live Search (live = 1), Limit References = None
# ──────────────────────────────────────────────────────────────────────────────

response_13, references_13 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", live=1, limit_references=None)
print(f"🤔 Response: {response_13}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_13}")

🤔 Response: 

A soldering iron is a hand-held tool used to melt solder, allowing you to join electronic components together. It consists of a heated metal tip that transfers heat to the solder. 🌟

When using a soldering iron with microcontrollers, follow these steps:

1. **Heat the Iron**: Turn on the soldering iron and let it heat up (around 300-400°C is ideal). 🔥
2. **Prepare the Components**: Insert the component leads through the PCB holes. 
3. **Apply Solder**: Touch the soldering iron tip to both the component lead and the PCB pad, then feed solder into the joint. 
4. **Cool Down**: Allow the joint to cool naturally for a strong connection. ❄️

Always remember to clean the tip and use a stand for safety! 😊
✨--------------------------------------------------✨
🔗 References: [1] Article Title: Microcontroller Tutorial 5/5: Soldering and Programming. - Link: https://www.build-electronic-circuits.com/microcontroller-tutorial-part5/ - Article Description: Soldering The Board. To solder

In [52]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #20)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1423910
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #19) → Current End (Stage #20):
{'teach_tokens_diff': 0, 'query_tokens_diff': 1380}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #19)         52405       1422530
End (Stage #20)           52405       1423910
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 20),2925,5827,✅ Changed,✅ Changed
Prior vs End (19 → 20),0,1380,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 5827}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 1380}


In [53]:
# ──────────────────────────────────────────────────────────────────────────────
# Testing Live Search (live = 2), Limit References = None
# ──────────────────────────────────────────────────────────────────────────────

response_14, references_14 = ask_sage_question(message=examples_prompt, system_prompt="You are a soldering iron expert. You respond in short nice responses as to emulate a helpful assistant. Make sure to be kind and informative in the responses. Use emojis and symbols to make the response more friendly. Do not use any other words that are not related to the question.", live=2, limit_references=None)
print(f"🤔 Response: {response_14}")
print('✨--------------------------------------------------✨')
print(f"🔗 References: {references_14}")

🤔 Response: 

A soldering iron is a hand tool used to heat solder, allowing you to join electronic components together. It's essential for creating strong electrical connections in microcontroller projects! 🌟

### How to Use a Soldering Iron with Microcontrollers:
1. **Prepare Your Work Area**: Ensure it's clean, well-lit, and ventilated. 🧹💡
2. **Clean and Tin the Tip**: Wipe the tip with a sponge or brass wool, then apply solder to improve heat transfer. ✨
3. **Position Components**: Insert component leads into the circuit board and bend them to hold in place. 🔌
4. **Heat and Solder**: Touch the soldering iron to the lead and pad, then apply solder. Allow it to cool. 🔥➡️❄️
5. **Inspect the Joint**: Ensure it's shiny and smooth; trim excess leads if needed. 🔍✂️
6. **Repeat for All Components**: Make sure all connections are properly soldered! 🔄

Happy soldering! 🎉
✨--------------------------------------------------✨
🔗 References: [1] Article Title: Microcontroller Tutorial 5/5: Solderi

In [54]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #21)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1425435
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #20) → Current End (Stage #21):
{'teach_tokens_diff': 0, 'query_tokens_diff': 1525}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #20)         52405       1423910
End (Stage #21)           52405       1425435
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 21),2925,7352,✅ Changed,✅ Changed
Prior vs End (20 → 21),0,1525,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 7352}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 1525}


----------------------------


### Tools and Tool Choice

#### Understanding `tools` vs `tool_choice` with Ask Sage API

When using the Ask Sage API, you have two important parameters to consider: `tools` and `tool_choice`. They serve different purposes and can significantly affect how your model interacts with the user prompt.
- **`tools`**: A list of available functions (tools) that the model can use if it thinks they are helpful.
- **`tool_choice`**: A specific instruction that forces the model to call a specific tool with specific arguments.
---

##### 1. What is `tools`?

- **Definition**:  
  The `tools` parameter is a **list of available functions** (tools) that you offer to the LLM.

- **Purpose**:  
  It tells the model:  
  > "These are the tools you are allowed to use, if you feel they help answer the user's prompt. Otherwise, just respond normally."

- **Format**:
  ```json
  [
    {
      "type": "function",
      "function": {
        "name": "calculate_bmi",
        "description": "Calculates BMI",
        "parameters": {
          "type": "object",
          "properties": {
            "weight_kg": { "type": "number", "description": "Weight in kilograms" },
            "height_cm": { "type": "number", "description": "Height in centimeters" }
          },
          "required": ["weight_kg", "height_cm"]
        }
      }
    }
  ]
  ```

- **Behavior**:
  - The LLM **analyzes** the user prompt.
  - It **decides by itself**:
    - Should it use a tool?
    - If yes, which tool and what arguments?
    - If no, just respond normally.

---

##### 2. What is `tool_choice`?

- **Definition**:  
  The `tool_choice` parameter is a **specific instruction** that forces the model to call a specific tool with specific arguments.

- **Purpose**:  
  It tells the model:  
  > "Don't think. Immediately call this specific tool, with these specific inputs."

- **Format**:
  ```json
  {
    "type": "function",
    "function": {
      "name": "calculate_bmi",
      "arguments": "{\"weight_kg\": 75, \"height_cm\": 180}"
    }
  }
  ```

- **Behavior**:
  - The model **does not reason**.
  - It **must** call exactly the tool you specify.
  - It **must** use exactly the arguments you provide.

---

##### Visual Mental Model

| Concept       | Think of it like...                           | Who decides the tool? | Who fills in the arguments? |
|---------------|------------------------------------------------|-----------------------|------------------------------|
| `tools`       | A toolbox 🧰 you hand to the model, saying "Use any tool you need" | The model chooses      | The model decides            |
| `tool_choice` | A specific tool 🛠️ you pick from the toolbox and say "Use this tool exactly like this" | You choose             | You provide all arguments    |


---

##### Quick Real-World Example

Suppose the user says:  
> "How old is someone born in 1990?"

- **Only using `tools`**:
  - Model **chooses** if it should call `calculate_age`.
  - It **figures out** `{birth_year: 1990}`.
  - It might even decide **not to** call a tool and just answer.

- **Using `tool_choice`**:
  - You **force** the model:
    - "Call `calculate_age`."
    - "Use `{birth_year: 1990}`."
  - No thinking, no guessing.

---

##### Why Does This Matter?

| Use `tools` when...                         | Use `tool_choice` when...                        |
|----------------------------------------------|--------------------------------------------------|
| You want **flexible**, **natural** AI behavior | You want **total control**                      |
| Let the model **reason about** what is best    | Force the model to **skip thinking**             |
| The user's prompt might need interpretation    | You already **know exactly** which function to call |
| You're building a **general chatbot**          | You're building a **specific app** (e.g., BMI calculator button) |

---

##### Sample Code Side-by-Side

###### Only using `tools`:

```python
response = ask_sage_client.query(
    message="Calculate BMI for 75kg and 180cm",
    tools=available_tools
)
```
The model reads the prompt and **decides** if it should use a tool.

---

###### Using `tools` + `tool_choice`:

```python
tool_choice = {
    "type": "function",
    "function": {
        "name": "calculate_bmi",
        "arguments": json.dumps({
            "weight_kg": 75,
            "height_cm": 180
        })
    }
}

response = ask_sage_client.query(
    message="Calculate BMI for 75kg and 180cm",
    tools=available_tools,
    tool_choice=tool_choice
)
```
The model **must** call `calculate_bmi` with the exact parameters.

---

#### Final Quick Summary

| Term          | Summary                                          |
|---------------|--------------------------------------------------|
| `tools`       | "Here's a list of tools you can use if you want." |
| `tool_choice` | "You MUST use *this* tool with *these* inputs."   |

---



### Tools - Example
![Description of the image](/Users/mark.espinozaasksage.ai/Code/AskSage-Open-Source-Community/images/tool_calling.png)

| Section                | Details                                                                                          |
|-----------------------|--------------------------------------------------------------------------------------------------|
| User Input            | A natural language message (e.g., "Calculate BMI...")                                          |
| Run Query (run_query) | Sends the message + available tools to the AskSage API                                         |
| Available Tools       | 14 tools built dynamically using `build_tool()`                                                |
| AskSage Decision      | Decides if a tool call is needed - Includes tool call details in response                      |
| Process Response      | Checks if tool calls exist                                                                       |
| Tool Execution        | - Parses the tool call <br> - Finds corresponding Python function in `tool_execution_map` <br> - Executes with the given parameters |
| Return Outputs        | Structured result (tool result) or plain chat message                                           |



In [55]:
pp = pprint.PrettyPrinter(indent=2)  # Create a PrettyPrinter instance with an indent of 2 spaces for better readability.

# === TOOL BUILDER ===
def build_tool(name, description, parameters, required):
    """
    Builds a tool definition in the format expected by the language model.

    Args:
        name (str): The name of the tool.
        description (str): A description of what the tool does.
        parameters (dict): A dictionary defining the parameters the tool accepts.
                         Each parameter is defined with a 'type' and 'description'.
        required (list): A list of parameter names that are required.

    Returns:
        dict: A dictionary representing the tool, ready to be used by the language model.
    """
    return {
        "type": "function",  # Specifies that this is a function-based tool.
        "function": {
            "name": name,  # The name of the function.
            "description": description,  # A description of what the function does.
            "parameters": {
                "type": "object",  # The parameters are defined as a JSON object.
                "properties": parameters,  # The properties of the object, defining each parameter.
                "required": required  # A list of required parameter names.
            }
        }
    }

# === TOOL DEFINITIONS ===
available_tools = [
    build_tool("calculate_bmi", "Calculates Body Mass Index (BMI)",
               {
                   "weight_kg": {"type": "number", "description": "Weight in kilograms"},
                   "height_cm": {"type": "number", "description": "Height in centimeters"}
               },
               ["weight_kg", "height_cm"]),

    build_tool("convert_celsius_to_fahrenheit", "Converts Celsius to Fahrenheit",
               {
                   "celsius": {"type": "number", "description": "Temperature in Celsius"}
               },
               ["celsius"]),

    build_tool("calculate_age", "Calculates age from birth year",
               {
                   "birth_year": {"type": "number", "description": "Birth year"}
               },
               ["birth_year"]),

    build_tool("is_prime", "Checks if a number is prime",
               {
                   "number": {"type": "number", "description": "Number to check"}
               },
               ["number"]),

    build_tool("calculate_calories_burned", "Estimates calories burned",
               {
                   "weight_kg": {"type": "number", "description": "Weight in kilograms"},
                   "met_value": {"type": "number", "description": "MET value of activity"},
                   "duration_minutes": {"type": "number", "description": "Duration in minutes"}
               },
               ["weight_kg", "met_value", "duration_minutes"]),

    build_tool("recommend_workout", "Suggests workout based on goal",
               {
                   "goal": {"type": "string", "description": "Fitness goal"}
               },
               ["goal"]),

    build_tool("estimate_vo2_max", "Estimates VO2 max from run data",
               {
                   "time_minutes": {"type": "number", "description": "Time in minutes"},
                   "distance_km": {"type": "number", "description": "Distance in kilometers"}
               },
               ["time_minutes", "distance_km"]),

    build_tool("calculate_target_heart_rate", "Calculates target heart rate",
               {
                   "age": {"type": "number", "description": "Age in years"}
               },
               ["age"]),

    build_tool("calculate_one_rep_max", "Estimates 1-rep max from weight and reps",
               {
                   "weight_kg": {"type": "number", "description": "Weight lifted"},
                   "reps": {"type": "number", "description": "Number of reps"}
               },
               ["weight_kg", "reps"]),

    build_tool("calculate_macronutrient_needs", "Calculates daily macronutrient needs",
               {
                   "calories": {"type": "number", "description": "Daily caloric intake"},
                   "goal": {"type": "string", "description": "Goal like 'muscle gain'"}
               },
               ["calories", "goal"]),

    build_tool("suggest_step_goal", "Suggests daily step goal based on activity",
               {
                   "activity_level": {"type": "string", "description": "Activity level (e.g., 'moderate')"}
               },
               ["activity_level"]),

    build_tool("calculate_water_intake", "Calculates recommended daily water intake",
               {
                   "weight_kg": {"type": "number", "description": "Weight in kilograms"}
               },
               ["weight_kg"]),

    build_tool("recommend_rest_days", "Recommends rest days based on workout frequency and intensity",
               {
                   "workouts_per_week": {"type": "number", "description": "Number of workouts per week"},
                   "intensity_level": {"type": "string", "description": "Intensity level (e.g., 'high')"}
               },
               ["workouts_per_week", "intensity_level"]),

    build_tool("create_plotly_chart", "Creates a basic Plotly bar chart",
               {
                   "labels": {"type": "array", "items": {"type": "string"}, "description": "List of labels"},
                   "values": {"type": "array", "items": {"type": "number"}, "description": "Corresponding values"},
                   "title": {"type": "string", "description": "Chart title"}
               },
               ["labels", "values", "title"])
]

# === TOOL EXECUTION FUNCTIONS ===
def calculate_bmi(args):
    """
    Calculates the Body Mass Index (BMI).

    Args:
        args (dict): A dictionary containing 'weight_kg' (weight in kilograms) and 'height_cm' (height in centimeters).

    Returns:
        float: The calculated BMI, rounded to two decimal places.
    """
    return round(args["weight_kg"] / ((args["height_cm"] / 100) ** 2), 2)

def convert_celsius_to_fahrenheit(args):
    """
    Converts Celsius to Fahrenheit.

    Args:
        args (dict): A dictionary containing 'celsius' (temperature in Celsius).

    Returns:
        float: The converted temperature in Fahrenheit, rounded to two decimal places.
    """
    return round((args["celsius"] * 9/5) + 32, 2)

def calculate_age(args):
    """
    Calculates age based on the birth year.

    Args:
        args (dict): A dictionary containing 'birth_year' (the year of birth).

    Returns:
        int: The calculated age.
    """
    return datetime.now().year - args["birth_year"]

def is_prime(args):
    """
    Checks if a number is prime.

    Args:
        args (dict): A dictionary containing 'number' (the number to check).

    Returns:
        bool: True if the number is prime, False otherwise.
    """
    n = args["number"]
    if n <= 1:
        return False
    return all(n % i for i in range(2, int(n**0.5) + 1))

def calculate_calories_burned(args):
    """
    Calculates the estimated calories burned during an activity.

    Args:
        args (dict): A dictionary containing 'weight_kg' (weight in kilograms), 'met_value' (MET value of the activity),
                     and 'duration_minutes' (duration of the activity in minutes).

    Returns:
        float: The estimated calories burned, rounded to two decimal places.
    """
    return round(args["met_value"] * args["weight_kg"] * (args["duration_minutes"] / 60), 2)

def recommend_workout(args):
    """
    Recommends a workout plan based on the fitness goal.

    Args:
        args (dict): A dictionary containing 'goal' (the fitness goal).

    Returns:
        str: A recommended workout plan based on the goal.
    """
    goal = args["goal"].lower()
    if "weight" in goal:
        return "HIIT + Strength Training 3x/week"
    if "muscle" in goal:
        return "Heavy Lifting 4x/week"
    if "endurance" in goal:
        return "Running + Cross-training 5x/week"
    return "Mixed training 3-4x/week"

def estimate_vo2_max(args):
    """
    Estimates VO2 max based on running data.

    Args:
        args (dict): A dictionary containing 'time_minutes' (time in minutes) and 'distance_km' (distance in kilometers).

    Returns:
        float: The estimated VO2 max, rounded to two decimal places.
    """
    return round(((args["distance_km"] * 1000) / args["time_minutes"]) * 0.2 + 3.5, 2)

def calculate_target_heart_rate(args):
    """
    Calculates the target heart rate zone based on age.

    Args:
        args (dict): A dictionary containing 'age' (age in years).

    Returns:
        str: A string describing the target heart rate zone.
    """
    max_hr = 220 - args["age"]
    return f"Target HR Zone: {int(max_hr*0.5)}-{int(max_hr*0.85)} bpm"

def calculate_one_rep_max(args):
    """
    Estimates the one-rep max (1RM) based on weight lifted and number of reps.

    Args:
        args (dict): A dictionary containing 'weight_kg' (weight lifted in kilograms) and 'reps' (number of repetitions).

    Returns:
        float: The estimated one-rep max, rounded to two decimal places.
    """
    return round(args["weight_kg"] * (1 + args["reps"] / 30), 2)

def calculate_macronutrient_needs(args):
    """
    Calculates the daily macronutrient needs based on caloric intake and fitness goal.

    Args:
        args (dict): A dictionary containing 'calories' (daily caloric intake) and 'goal' (fitness goal).

    Returns:
        dict: A dictionary containing the recommended grams of protein, fat, and carbohydrates.
    """
    goal = args["goal"].lower()
    if "muscle" in goal:
        p, f, c = 0.35, 0.25, 0.4
    elif "weight" in goal:
        p, f, c = 0.4, 0.3, 0.3
    else:
        p, f, c = 0.3, 0.25, 0.45
    return {
        "Protein (g)": round((args["calories"] * p) / 4),
        "Fat (g)": round((args["calories"] * f) / 9),
        "Carbs (g)": round((args["calories"] * c) / 4)
    }

def suggest_step_goal(args):
    """
    Suggests a daily step goal based on activity level.

    Args:
        args (dict): A dictionary containing 'activity_level' (activity level, e.g., 'moderate').

    Returns:
        str: A suggested daily step goal.
    """
    level = args["activity_level"].lower()
    return {
        "sedentary": "5000 steps/day",
        "moderate": "7500 steps/day",
        "active": "10000+ steps/day"
    }.get(level, "7000-10000 steps/day")

def calculate_water_intake(args):
    """
    Calculates the recommended daily water intake based on weight.

    Args:
        args (dict): A dictionary containing 'weight_kg' (weight in kilograms).

    Returns:
        str: A string describing the recommended daily water intake in liters.
    """
    return f"{round(args['weight_kg'] * 0.033, 2)} liters/day"

def recommend_rest_days(args):
    """
    Recommends the number of rest days based on workout frequency and intensity.

    Args:
        args (dict): A dictionary containing 'workouts_per_week' (number of workouts per week) and
                     'intensity_level' (intensity level, e.g., 'high').

    Returns:
        str: A string describing the recommended number of rest days per week.
    """
    intensity = args["intensity_level"].lower()
    workouts = args["workouts_per_week"]
    if "high" in intensity:
        return f"{2 if workouts >= 5 else 1} rest days/week"
    if "moderate" in intensity:
        return f"{1 if workouts >= 4 else 2} rest days/week"
    return "1 rest day/week"

def create_plotly_chart(args):
    """
    Creates and displays a basic Plotly bar chart.

    Args:
        args (dict): A dictionary containing 'labels' (list of labels), 'values' (list of corresponding values),
                     and 'title' (chart title).

    Returns:
        str: A message indicating that the chart has been displayed.
    """
    fig = go.Figure([go.Bar(x=args["labels"], y=args["values"])])  # Create a bar chart figure.
    fig.update_layout(title=args["title"], xaxis_title="Labels", yaxis_title="Values")  # Update the layout with title and axis labels.
    fig.show()  # Display the chart.
    return "Chart displayed successfully."


# Map tool names to execution handlers
tool_execution_map = {
    "calculate_bmi": calculate_bmi,
    "convert_celsius_to_fahrenheit": convert_celsius_to_fahrenheit,
    "calculate_age": calculate_age,
    "is_prime": is_prime,
    "calculate_calories_burned": calculate_calories_burned,
    "recommend_workout": recommend_workout,
    "estimate_vo2_max": estimate_vo2_max,
    "calculate_target_heart_rate": calculate_target_heart_rate,
    "calculate_one_rep_max": calculate_one_rep_max,
    "calculate_macronutrient_needs": calculate_macronutrient_needs,
    "suggest_step_goal": suggest_step_goal,
    "calculate_water_intake": calculate_water_intake,
    "recommend_rest_days": recommend_rest_days,
    "create_plotly_chart": create_plotly_chart
}

# === TOOL EXECUTOR ===
def execute_tool_call(tool_call):
    """
    Executes a tool call based on the provided tool call definition.

    Args:
        tool_call (dict): A dictionary containing the tool's function name and arguments.

    Returns:
        str or any: The result of the tool execution, or an error message if something goes wrong.
    """
    try:
        function_name = tool_call['function']['name']  # Extract the function name from the tool call.
        arguments = json.loads(tool_call['function']['arguments'])  # Extract the arguments and parse them from JSON.
        handler = tool_execution_map.get(function_name)  # Get the corresponding function handler from the map.
        if handler:
            return handler(arguments)  # Execute the handler with the arguments and return the result.
        return f"Unknown function: {function_name}"  # If the function is unknown, return an error message.
    except KeyError as e:
        return f"Missing parameter: {e}"  # If a required parameter is missing, return an error message.
    except Exception as e:
        return f"Execution error: {e}"  # If any other exception occurs during execution, return an error message.

# === RUNNING QUERY TO REAL API ===
def run_query(message, model="gpt-4o-mini", system_prompt="Tool Test"):
    """
    Runs a query against the language model API.

    Args:
        message (str): The query message.
        model (str): The model to use (default: "gpt-4o-mini").
        system_prompt (str): The system prompt to use (default: "Tool Test").

    Returns:
        dict: The response from the language model API.
    """
    return ask_sage_client.query(
        message=message,
        model=model,
        limit_references=0,
        system_prompt=system_prompt,
        tools=available_tools
    )

# === PROCESS RESPONSE ===
def process_response(response):
    """
    Processes the response from the language model API.

    Args:
        response (dict): The response from the language model API.

    Returns:
        list: A list of outputs, each indicating either a tool call result or a chat response.
    """
    outputs = []

    if response.get('tool_calls'):  # Check if the response contains tool calls.
        # print entire response for debugging
        # pp.pprint(response)
        for call in response['tool_calls']:  # Iterate through each tool call.
            result = execute_tool_call(call)  # Execute the tool call and get the result.
            outputs.append({
                "type": "tool_call",  # Indicate that this output is a tool call result.
                "tool_name": call['function']['name'],  # Store the name of the tool.
                "result": result  # Store the result of the tool execution.
            })
    else:
        outputs.append({
            "type": "chat_response",  # Indicate that this output is a chat response.
            "message": response.get('message', 'No message found.')  # Store the chat message, or a default message if none is found.
        })

    return outputs

# === RUN A FEW EXAMPLES AND SAVE TO DATAFRAME WITH TIMING ===
def run_examples_to_dataframe():
    """
    Runs a series of example prompts, prints live results, measures timings, and collects into a dataframe.
    """
    examples = [
        ("Calculate BMI for someone who weighs 75 kg and is 180 cm tall", "BMI (Metric)"),
        ("Calculate BMI for someone who weighs 165 pounds and is 70 inches tall", "BMI (Imperial)"),
        ("How much water should a 70 kg person drink daily?", "Water Intake (Metric)"),
        ("How much water should a 154 pound person drink daily?", "Water Intake (Imperial)"),
        ("How old is someone born in 1990?", "Age Calculation"),
        ("What is the target heart rate for a 30-year-old?", "Target Heart Rate"),
        ("Estimate one-rep max for lifting 100 kg for 5 reps", "One-Rep Max (Metric)"),
        ("Estimate one-rep max for lifting 220 pounds for 5 reps", "One-Rep Max (Imperial)"),
        ("How many calories does a 70 kg person burn jogging for 30 minutes at MET 7?", "Calories Burned (Metric)"),
        ("How many calories does a 154 pound person burn jogging for 30 minutes at MET 7?", "Calories Burned (Imperial)"),
        ("Suggest a workout plan for someone who wants to gain muscle", "Workout Recommendation"),
        ("Estimate VO2 max for running 2.4 kilometers in 12 minutes", "VO2 Max Estimation"),
        ("Suggest a daily step goal for someone moderately active", "Step Goal Suggestion"),
        ("How many rest days are recommended for someone who works out 5 days a week at high intensity?", "Rest Days Recommendation"),
        ("Calculate macronutrient needs for a 2500 calorie diet aimed at muscle gain", "Macronutrient Needs"),
        ("Who was the first president of the USA?", "General Chat (No Tool)"),
        ("What is the capital of France?", "General Chat (No Tool)"),
        ("What's the weather like today?", "General Chat (No Tool)"),
        ("How far is the Moon from Earth?", "General Chat (No Tool)"),
        ("Is 29 a prime number?", "Prime Check"),
        ("Convert 30 degrees Celsius to Fahrenheit", "Temperature Conversion"),
        ("Create a chart showing ['Running', 'Cycling', 'Swimming'] with times [30, 45, 25] titled 'Weekly Exercise Time'", "Create Plotly Chart"),
        ("Please calculate the ingredients for a perfect espresso-based drink.", "Espresso Ingredients Calculation"),
    ]

    records = []  # List to store all the records

    for idx, (prompt_text, description) in enumerate(examples, start=1):
        print(f"\n=== Example {idx}: {description} ===")
        print(f"Prompt: {prompt_text}")
        
        start_time = datetime.now()
        start_timestamp = start_time.strftime("%Y-%m-%d %H:%M:%S")
        
        try:
            response = run_query(prompt_text)
            results = process_response(response)
            status = "Success"
        except Exception as e:
            results = [{"type": "error", "message": str(e)}]
            status = "Error"
        
        end_time = datetime.now()
        end_timestamp = end_time.strftime("%Y-%m-%d %H:%M:%S")
        elapsed_seconds = round((end_time - start_time).total_seconds(), 2)

        # Print each result live
        for res in results:
            print(res)
            
            record = {
                "Example #": idx,
                "Prompt": prompt_text,
                "Description": description,
                "Result Type": res.get("type"),
                "Tool Name": res.get("tool_name", None),
                "Result / Message": res.get("result", res.get("message")),
                "Status": status,
                "Start Time": start_timestamp,
                "End Time": end_timestamp,
                "Elapsed Seconds": elapsed_seconds
            }
            records.append(record)

        print(f"⏱️ Took {elapsed_seconds} seconds")
        print("-" * 50)

    df = pd.DataFrame(records)
    return df

# === TRIGGER ===
example_run_df = run_examples_to_dataframe()

# Display the DataFrame
display(example_run_df)  # Display the DataFrame in a readable format



=== Example 1: BMI (Metric) ===
Prompt: Calculate BMI for someone who weighs 75 kg and is 180 cm tall
{'type': 'tool_call', 'tool_name': 'calculate_bmi', 'result': 23.15}
⏱️ Took 1.04 seconds
--------------------------------------------------

=== Example 2: BMI (Imperial) ===
Prompt: Calculate BMI for someone who weighs 165 pounds and is 70 inches tall
{'type': 'tool_call', 'tool_name': 'calculate_bmi', 'result': 23.67}
⏱️ Took 1.51 seconds
--------------------------------------------------

=== Example 3: Water Intake (Metric) ===
Prompt: How much water should a 70 kg person drink daily?
{'type': 'tool_call', 'tool_name': 'calculate_water_intake', 'result': '2.31 liters/day'}
⏱️ Took 1.23 seconds
--------------------------------------------------

=== Example 4: Water Intake (Imperial) ===
Prompt: How much water should a 154 pound person drink daily?
{'type': 'tool_call', 'tool_name': 'calculate_water_intake', 'result': '2.31 liters/day'}
⏱️ Took 0.97 seconds
-----------------------

{'type': 'tool_call', 'tool_name': 'create_plotly_chart', 'result': 'Chart displayed successfully.'}
⏱️ Took 2.16 seconds
--------------------------------------------------

=== Example 23: Espresso Ingredients Calculation ===
Prompt: Please calculate the ingredients for a perfect espresso-based drink.
{'type': 'chat_response', 'message': "\n\nTo create a perfect espresso-based drink, you'll typically need the following ingredients:\n\n1. **Espresso**: \n   - Freshly ground coffee beans (about 18-20 grams for a double shot)\n   - Water (about 30-40 ml for a double shot)\n\n2. **Milk** (if making a milk-based drink like a latte or cappuccino):\n   - Whole milk or your choice of milk (about 150-200 ml, depending on the drink)\n\n3. **Flavoring (optional)**:\n   - Sugar or sweetener (to taste)\n   - Syrups (like vanilla, caramel, or hazelnut, if desired)\n\n4. **Toppings (optional)**:\n   - Cocoa powder or cinnamon (for dusting)\n   - Whipped cream (for certain drinks)\n\n### Basic Recipe

,Example #,Prompt,Description,Result Type,Tool Name,Result / Message,Status,Start Time,End Time,Elapsed Seconds
0,1,Calculate BMI for someone who weighs 75 kg and is 180 cm tall,BMI (Metric),tool_call,calculate_bmi,23.15,Success,2025-05-08 16:46:11,2025-05-08 16:46:12,1.04
1,2,Calculate BMI for someone who weighs 165 pounds and is 70 inches tall,BMI (Imperial),tool_call,calculate_bmi,23.67,Success,2025-05-08 16:46:12,2025-05-08 16:46:14,1.51
2,3,How much water should a 70 kg person drink daily?,Water Intake (Metric),tool_call,calculate_water_intake,2.31 liters/day,Success,2025-05-08 16:46:14,2025-05-08 16:46:15,1.23
3,4,How much water should a 154 pound person drink daily?,Water Intake (Imperial),tool_call,calculate_water_intake,2.31 liters/day,Success,2025-05-08 16:46:15,2025-05-08 16:46:16,0.97
4,5,How old is someone born in 1990?,Age Calculation,tool_call,calculate_age,35,Success,2025-05-08 16:46:16,2025-05-08 16:46:17,0.96
5,6,What is the target heart rate for a 30-year-old?,Target Heart Rate,tool_call,calculate_target_heart_rate,Target HR Zone: 95-161 bpm,Success,2025-05-08 16:46:17,2025-05-08 16:46:18,1.47
6,7,Estimate one-rep max for lifting 100 kg for 5 reps,One-Rep Max (Metric),tool_call,calculate_one_rep_max,116.67,Success,2025-05-08 16:46:18,2025-05-08 16:46:20,1.42
7,8,Estimate one-rep max for lifting 220 pounds for 5 reps,One-Rep Max (Imperial),tool_call,calculate_one_rep_max,116.42,Success,2025-05-08 16:46:20,2025-05-08 16:46:21,1.33
8,9,How many calories does a 70 kg person burn jogging for 30 minutes at MET 7?,Calories Burned (Metric),tool_call,calculate_calories_burned,245.0,Success,2025-05-08 16:46:21,2025-05-08 16:46:23,1.32
9,10,How many calories does a 154 pound person burn jogging for 30 minutes at MET 7?,Calories Burned (Imperial),tool_call,calculate_calories_burned,244.49,Success,2025-05-08 16:46:23,2025-05-08 16:46:24,1.85


In [56]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #22)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1425648
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #21) → Current End (Stage #22):
{'teach_tokens_diff': 0, 'query_tokens_diff': 213}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #21)         52405       1425435
End (Stage #22)           52405       1425648
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 22),2925,7565,✅ Changed,✅ Changed
Prior vs End (21 → 22),0,213,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 7565}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 213}


### Tool Choice - Example

In [57]:
# === NEW TOOL DEFINITION ===
calculate_espresso_ingredients_tool = {
    "type": "function",
    "function": {
        "name": "calculate_espresso_ingredients",
        "description": "Calculates the amount of coffee beans and milk needed for a perfect espresso-based drink.",
        "parameters": {
            "type": "object",
            "properties": {
                "espresso_shots": {
                    "type": "number",
                    "description": "Number of espresso shots (1 shot = 30ml)."
                },
                "milk_ratio": {
                    "type": "number",
                    "description": "Milk-to-espresso ratio (e.g., 2.0 means 2x milk volume relative to espresso volume)."
                }
            },
            "required": ["espresso_shots", "milk_ratio"]
        }
    }
}

# === NEW TOOL EXECUTION FUNCTION ===
def calculate_espresso_ingredients(args):
    espresso_shots = args["espresso_shots"]
    milk_ratio = args["milk_ratio"]
    
    espresso_volume_ml = espresso_shots * 30  # 1 shot = 30ml
    milk_volume_ml = espresso_volume_ml * milk_ratio
    
    return {
        "espresso_volume_ml": espresso_volume_ml,
        "milk_volume_ml": round(milk_volume_ml, 2)
    }

# Add new tool to available tools and execution map
updated_tools = available_tools + [calculate_espresso_ingredients_tool]
tool_execution_map["calculate_espresso_ingredients"] = calculate_espresso_ingredients

# === TOOL CHOICE you are forcing ===
tool_choice = {
    "type": "function",
    "function": {
        "name": "calculate_espresso_ingredients",
        "arguments": json.dumps({
            "espresso_shots": 2,
            "milk_ratio": 2.0
        })
    }
}

# === RUN TOOL CHOICE EXAMPLE ===
def run_tool_choice_example():
    """
    Runs a dedicated example forcing a specific tool choice, prints live, and collects timing into a dataframe.
    """
    prompt_text = "Please calculate the ingredients for a perfect espresso-based drink."
    description = "Espresso Ingredients Calculation (Forced Tool Choice)"

    print(f"\n=== Tool Choice Example ===")
    print(f"Prompt: {prompt_text}")

    start_time = datetime.now()
    start_timestamp = start_time.strftime("%Y-%m-%d %H:%M:%S")

    try:
        response = ask_sage_client.query(
            message=prompt_text,
            model="gpt-4o-mini",
            limit_references=0,
            system_prompt="Tool Test - Espresso Ingredients Calculation",
            tools=updated_tools,
            tool_choice=tool_choice  # <-- FORCING the tool
        )
        results = process_response(response)
        status = "Success"
    except Exception as e:
        results = [{"type": "error", "message": str(e)}]
        status = "Error"

    end_time = datetime.now()
    end_timestamp = end_time.strftime("%Y-%m-%d %H:%M:%S")
    elapsed_seconds = round((end_time - start_time).total_seconds(), 2)

    records = []

    for res in results:
        print(res)

        record = {
            "Prompt": prompt_text,
            "Description": description,
            "Result Type": res.get("type"),
            "Tool Name": res.get("tool_name", None),
            "Result / Message": res.get("result", res.get("message")),
            "Status": status,
            "Start Time": start_timestamp,
            "End Time": end_timestamp,
            "Elapsed Seconds": elapsed_seconds
        }
        records.append(record)

    print(f"⏱️ Took {elapsed_seconds} seconds")
    print("-" * 50)

    df = pd.DataFrame(records)
    return df

# === TRIGGER ===
tool_choice_df = run_tool_choice_example()

# Display the DataFrame
display(tool_choice_df)  # Display the DataFrame in a readable format



=== Tool Choice Example ===
Prompt: Please calculate the ingredients for a perfect espresso-based drink.
{'type': 'tool_call', 'tool_name': 'calculate_espresso_ingredients', 'result': {'espresso_volume_ml': 30, 'milk_volume_ml': 60.0}}
⏱️ Took 1.09 seconds
--------------------------------------------------


,Prompt,Description,Result Type,Tool Name,Result / Message,Status,Start Time,End Time,Elapsed Seconds
0,Please calculate the ingredients for a perfect espresso-based drink.,Espresso Ingredients Calculation (Forced Tool Choice),tool_call,calculate_espresso_ingredients,"{'espresso_volume_ml': 30, 'milk_volume_ml': 60.0}",Success,2025-05-08 16:46:50,2025-05-08 16:46:51,1.09


In [58]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #23)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1425658
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #22) → Current End (Stage #23):
{'teach_tokens_diff': 0, 'query_tokens_diff': 10}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #22)         52405       1425648
End (Stage #23)           52405       1425658
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 23),2925,7575,✅ Changed,✅ Changed
Prior vs End (22 → 23),0,10,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 7575}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 10}


## 15. Query with file

This endpoint makes it easy to pass a prompt with the context of a file. The file will be appended to the prompt and the model will be able to use the prompt and context from the file to generate a response.

Note: The query_with_file works similar to the query endpoint, but it allows you to pass a file as context. The file will be appended to the prompt and the model will be able to use the prompt and context from the file to generate a response.

Here are the parameters for the `query_with_file` endpoint:

```python

query_with_file(self, message, file=None, persona='default', dataset='all', limit_references=None, temperature=0.0, live=0, model='openai_gpt', tools=None, tool_choice=None)

```

`file` parameter is the path to the file that will be appended to the prompt.


<div class="alert alert-block alert-warning">
<b>Note:</b> Passing an entire file as context is not the same as using a dataset. The difference is the entire file is passed as context and the model will be able to use the entire file to generate a response. The dataset is used to perform RAG(Retrieval-Augmented Generation) where the model retrieves information from the dataset to generate responses leveraging sophisticated semantic search capabilities.

Limitations with the `query_with_file` endpoint:
- It will use more tokens since the entire file is passed as context.

</div>

In [59]:
# Data path
file_path = 'data/query_with_file/'

# Get files in the data path
files = os.listdir(file_path)

message = 'Would Toby get along with Whiskers? - keep it short use only 10 tokens'

# Query with file
query_with_file_data = ask_sage_client.query_with_file(
    message=message,
    file=file_path + files[4],
    dataset='none',
    limit_references=0
)

print("Query with file data:")
print(query_with_file_data['message'])


Query with file data:
<file-content>

{"asksage_metadata": {"filename": "In the small village of Willowbrook.pdf", "page_number": 1}}
In the small village of Willowbrook, there lived a scruffy little dog named Toby who had an unusual dislike for cats. Toby, with his ruffled fur and eager eyes, was known around the village for his playful spirit, but he would always steer clear of any feline. One day, a new cat moved into the neighborhood. The cat, sleek and confident, quickly noticed Toby's aversion. Curious and a bit mischievous, the cat decided to find out why Toby was so hesitant around its kind. This set the stage for a series of amusing encounters, as the cat tried to win over the reluctant Toby, leading to unexpected friendship and adventures that would change Toby's mind about cats forever.

</file-content>

Toby and Whiskers would eventually become unexpected friends.


In [60]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #24)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1425736
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #23) → Current End (Stage #24):
{'teach_tokens_diff': 0, 'query_tokens_diff': 78}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #23)         52405       1425658
End (Stage #24)           52405       1425736
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 24),2925,7653,✅ Changed,✅ Changed
Prior vs End (23 → 24),0,78,❌ No Change,✅ Changed


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 7653}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 78}


## 20. Delete Dataset

This endpoint is used to delete a dataset from the user's account. The only parameter required for this endpoint is the 'dataset', which specifies the specific dataset to delete. 

Notice how the dataset is no longer in the list of available datasets after deletion.

<div class="alert alert-block alert-warning">
<b>Note:</b> Running the cell below will clear the dataset from your account.
</div>


In [61]:
# Delete the two datasets that were created in the beginning
delete_dataset_1 = ask_sage_client.delete_dataset(dataset_entry_1) # Delete the first dataset
print(f"API response for deleting dataset {dataset_entry_1}: {delete_dataset_1}") # Print the API response for deleting the first dataset
delete_dataset_2 = ask_sage_client.delete_dataset(dataset_entry_2) # Delete the second dataset
print(f"API response for deleting dataset {dataset_entry_2}: {delete_dataset_2}") # Print the API response for deleting the second dataset

display_datasets(ask_sage_client)

API response for deleting dataset user_custom_34125_1-jupynotebk-asksage-ex-52_content: {'response': 'OK', 'status': 200}
API response for deleting dataset user_custom_34125_2-jupynotebk-asksage-ex-9_content: {'response': 'OK', 'status': 200}


,0
0,Acquisition.gov
1,Air Force
2,DoD
3,Department of Defense
4,Learn with Nic
5,In the Nic of Time
6,Platform One
7,Nic Chaillan's Website
8,Cloud One
9,NIST_NVD_CVE


In [62]:
print('✨--------------------------------------------------✨')
print_token_volume_report(ask_sage_client)

✨--------------------------------------------------✨

📊 Token Volume Report 📊 (Stage #25)
--------------------------------------------------
🔹 Reusing token volume from script start.
The count of monthly teach/training tokens used are: 52405
The count of monthly inference/query tokens used are: 1425736
🔹 Token volume recorded *after current script execution*.
--------------------------------------------------
🔹 Change from Prior End (Stage #24) → Current End (Stage #25):
{'teach_tokens_diff': 0, 'query_tokens_diff': 0}
🔹 Token Usage Table:
                   teach_tokens  query_tokens
Start (Stage #1)          49480       1418083
Prior (Stage #24)         52405       1425736
End (Stage #25)           52405       1425736
--------------------------------------------------
📈 Token Volume Comparison Flags:


,teach_tokens_diff,query_tokens_diff,teach_tokens_flag,query_tokens_flag
Start vs End (1 → 25),2925,7653,✅ Changed,✅ Changed
Prior vs End (24 → 25),0,0,❌ No Change,❌ No Change


--------------------------------------------------
📦 Total Token Volume Used Since Script Start:
{'teach_tokens_used': 2925, 'query_tokens_used': 7653}
📦 Token Volume Used in This Stage Only:
{'teach_tokens_used_this_stage': 0, 'query_tokens_used_this_stage': 0}
